# Importing some packages

In [1]:
import numpy as np
from numba import njit
import pandas as pd
from pathlib import Path
from collections import defaultdict
from tqdm import tqdm
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

# Limited testing

In [2]:
@njit(cache=True)
def get_signals(signal_list, exit_list):

    start_idx = 0
    exit_idx = 0
   
    for i in range(len(signal_list[0])):

        if i == start_idx:

            if signal_list[0][i] == 0:

                start_idx += 1

                exit_list[0][i] = 0

            else:

                for j in range(i+1, len(exit_list[0])):
                    if exit_list[0][j] == -signal_list[0][i]:
                        exit_idx = j
                        break
                    else:
                        exit_idx = j

                for k in range(i+1, exit_idx+1):
                    if signal_list[0][k] == -signal_list[0][i]:
                        exit_idx = k
                        exit_list[1][k] = signal_list[1][k]
                        break
                    else:
                        exit_idx = k

                for p in range(i+1, exit_idx):
                    signal_list[0][p] = 0
                    exit_list[0][p] = 0
                
                if exit_idx == len(signal_list[0]) - 1 and exit_list[0][exit_idx] != -signal_list[0][i]:
                    exit_list[0][exit_idx] = 0
                    exit_list[0][i] = 0
                    signal_list[0][exit_idx] = 0
                else:
                    exit_list[0][exit_idx] = -signal_list[0][i]
                    exit_list[0][i] = 0
                    signal_list[0][exit_idx] = 0
                
                start_idx = exit_idx + 1

        else:

            continue

    if sum(np.abs(signal_list[0])) == sum(np.abs(exit_list[0])):

        return signal_list, exit_list
    
    else:

        for i in range(len(signal_list[0])):
            if signal_list[0][-(i+1)] != 0:
                signal_list[0][-(i+1)] = 0
                break

        return signal_list, exit_list
    
@njit(cache=True)
def create_position_open_prices(signal_list, exit_list):

    pos_open_prices = np.zeros(len(signal_list[0]))
    pos_exit_prices = np.zeros(len(exit_list[0]))

    start_idx = 0
    price_idx = 0

    for i in range(len(signal_list[0])):
        if exit_list[0][i] != 0:
            for j in range(start_idx, i):
                if signal_list[0][j] != 0:
                    price_idx = j
                    break
            pos_open_prices[i] = signal_list[1][price_idx]
            pos_exit_prices[i] = exit_list[1][i]
            start_idx = i
        else:
            pass

    return pos_open_prices, pos_exit_prices

@njit(cache=True)
def get_pnl_testing(
    trade_close_prices,
    signal_list, 
    trade_open_prices,
    commission=0.015,
    slippage=0.05,
    init_inv=20000,
    trade_size=0.1
):

    pnl_list = np.zeros(len(trade_close_prices))

    for i in range(len(trade_close_prices)):

        if signal_list[i] == 0 or trade_open_prices[i] == 0:
            pass
        
        # signal_list contains the points where exit occurs
        elif signal_list[i] == -1: 
            temp_n_assets = int(init_inv * trade_size / trade_open_prices[i])
            temp_pnl = temp_n_assets * (trade_close_prices[i] - trade_open_prices[i] * (1 + slippage)) 
            temp_pnl = temp_pnl * (1 - commission)
            init_inv += temp_pnl

        else:
            temp_n_assets = int(init_inv * trade_size / trade_open_prices[i])
            temp_pnl = temp_n_assets * (trade_open_prices[i] * (1 - slippage) - trade_close_prices[i])
            temp_pnl = temp_pnl * (1 - commission)
            init_inv += temp_pnl

        pnl_list[i] = temp_pnl

    return pnl_list


## Entry testing

### Fixed stop and target exit

In [3]:
@njit(cache=True)
def get_exit_entry_testing1(
    close_prices, 
    open_prices,  
    signal_list,
    stoploss_th,
    takeprofit_th, 
    commission, 
    slippage, 
    init_inv, 
    trade_size
):

    exit_list = np.zeros((2, len(close_prices)))

    for i in range(len(close_prices)-1):

        if signal_list[1][i] == 0:

            pass

        else:

            temp_n_assets = int(init_inv * trade_size / signal_list[1][i])
            if signal_list[0][i] == 1:
                temp_pnl = temp_n_assets * (close_prices[i] - signal_list[1][i] * (1 + slippage))
            else:
                temp_pnl = -temp_n_assets * (close_prices[i] - signal_list[1][i] * (1 - slippage))
            temp_pnl = temp_pnl * (1 - commission)
            init_inv += temp_pnl

            if -temp_pnl >= stoploss_th or temp_pnl >= takeprofit_th:
                exit_list[0][i+1] = -signal_list[0][i]
                exit_list[1][i+1] = open_prices[i+1]
            else:
                pass
        
    return exit_list

### Fixed bar exit

In [4]:
@njit(cache=True)
def get_exit_entry_testing2( 
    open_prices,  
    signal_list,
    n_exit_bars
):

    exit_list = np.zeros((2, len(signal_list[0])))

    n_exit_bars = np.int64(n_exit_bars)

    for i in range(len(signal_list[0])-1):

        if signal_list[0][i] == 0:

            pass

        else:
            
            if i + n_exit_bars < len(signal_list[0]):
                exit_list[0][i+n_exit_bars] = -signal_list[0][i]
                exit_list[1][i+n_exit_bars] = open_prices[i+n_exit_bars]
            else:
                pass
        
    return exit_list


### Random exit

In [5]:
@njit(cache=True)
def get_exit_entry_testing3( 
    open_prices,  
    signal_list
):

    exit_list = np.zeros((2, len(signal_list[0])))

    for i in range(len(signal_list[0])-1):

        if signal_list[0][i] == 0:

            pass

        else:

            for j in range(i+1, len(signal_list[0])):
                if signal_list[0][j] != 0:
                    j = j - 1
                    break
                else:
                    if np.random.rand() > 0.5:
                        break
            
            exit_list[0][j] = -signal_list[0][i]
            exit_list[1][j] = open_prices[j]
        
    return exit_list


### Winning percentage

In [6]:
# def calculate_mean_win_perc_entry_testing(data, text_code):

#     bars_per_5week = 7 * 60 * 24 * 5
#     n_bars_per_year = 7 * 60 * 24 * 52

#     if data.shape[0] < n_bars_per_year:
#         n_bars_per_year = data.shape[0]

#     n_not_worked = 0
#     n_total_cases = 0

#     entry_walk_forward_dict = defaultdict(list)

#     for idx in range(0, n_bars_per_year, bars_per_5week):

#         n_total_cases += 1

#         df = data.iloc[idx:idx+bars_per_5week, :]
#         df.reset_index(drop=True, inplace=True)

#         price_data = {}
#         for col in df.columns:
#             if col == 'datetime':
#                 continue
#             else:
#                 price_data[col] = df[col].values
        
#         try:
#             exec_dict = {'price_data': price_data}
#             exec(text_code, exec_dict)

#             commission = exec_dict['COMMISSION']
#             slippage = exec_dict['SLIPPAGE'] 
#             init_inv = exec_dict['AVAILABLE_CAPITAL']
#             trade_size = exec_dict['TRADE_SIZE'] 

#             signal_idxs = list(exec_dict['buy_idxs'])
#             signal_idxs.extend(list(exec_dict['sell_idxs']))
#             signal_idxs = sorted(signal_idxs)
#             signal_idxs_true = [i - 1 for i in signal_idxs]

#             buy_idxs = [i - 1 for i in list(exec_dict['buy_idxs'])]
#             df['buy'] = 0
#             df.loc[df.index.isin(buy_idxs), 'buy'] = 1

#             sell_idxs = [i - 1 for i in list(exec_dict['sell_idxs'])]
#             df['sell'] = 0
#             df.loc[df.index.isin(sell_idxs), 'sell'] = -1

#             df['signal'] = df['buy'] + df['sell']

#             df['new_signal'] = 0
#             df.loc[df.index.isin(signal_idxs_true), 'new_signal'] = df.loc[df.index.isin(signal_idxs_true), 'signal'].values
#             df['signal_prices'] = 0
#             df.loc[df.index.isin(signal_idxs), 'signal_prices'] = df.loc[df.index.isin(signal_idxs), 'btc_open'].values

#             signal_list = np.zeros((2, df.shape[0]))
#             signal_list[0][1:] = df['new_signal'].values[:-1]
#             signal_list[1] = df['signal_prices'].values

#             # fixed stop and target exit testing
#             exit_list = get_exit_entry_testing1(
#                 close_prices=df['btc_close'].values, 
#                 open_prices=df['btc_open'].values,  
#                 signal_list=signal_list,
#                 stoploss_th=50,
#                 takeprofit_th=100,
#                 commission=commission, 
#                 slippage=slippage, 
#                 init_inv=init_inv, 
#                 trade_size=trade_size
#             )

#             signal_list, exit_list = get_signals(signal_list, exit_list)
#             pos_open_prices, pos_exit_prices = create_position_open_prices(signal_list, exit_list)

#             pnl_list = get_pnl_testing(
#                 trade_close_prices=pos_exit_prices,
#                 signal_list=exit_list[0], 
#                 trade_open_prices=pos_open_prices,
#                 commission=commission, 
#                 slippage=slippage, 
#                 init_inv=init_inv, 
#                 trade_size=trade_size
#             )

#             fixed_winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)
#             entry_walk_forward_dict['fixed_sp_testing'].append(fixed_winning_percent)

#             # fixed bar exit testing
#             exit_list = get_exit_entry_testing2( 
#                 open_prices=df['btc_open'].values,  
#                 signal_list=signal_list,
#                 n_exit_bars=5
#             )

#             signal_list, exit_list = get_signals(signal_list, exit_list)
#             pos_open_prices, pos_exit_prices = create_position_open_prices(signal_list, exit_list)

#             pnl_list = get_pnl_testing(
#                 trade_close_prices=pos_exit_prices,
#                 signal_list=exit_list[0], 
#                 trade_open_prices=pos_open_prices,
#                 commission=commission, 
#                 slippage=slippage, 
#                 init_inv=init_inv, 
#                 trade_size=trade_size
#             )

#             fixed_bar_winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)
#             entry_walk_forward_dict['fixed_bar_testing'].append(fixed_bar_winning_percent)

#             # random exit testing
#             exit_list = get_exit_entry_testing3( 
#                 open_prices=df['btc_open'].values,  
#                 signal_list=signal_list
#             )

#             signal_list, exit_list = get_signals(signal_list, exit_list)
#             pos_open_prices, pos_exit_prices = create_position_open_prices(signal_list, exit_list)

#             pnl_list = get_pnl_testing(
#                 trade_close_prices=pos_exit_prices,
#                 signal_list=exit_list[0], 
#                 trade_open_prices=pos_open_prices,
#                 commission=commission, 
#                 slippage=slippage, 
#                 init_inv=init_inv, 
#                 trade_size=trade_size
#             )

#             random_winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)
#             entry_walk_forward_dict['random_exit_testing'].append(random_winning_percent)

#         except:

#             n_not_worked += 1

#     mean_win_perc_dict = defaultdict(list)

#     mean_win_perc_dict['Fixed_StopLoss_TakeProfit_testing'].\
#         append(np.mean(entry_walk_forward_dict['fixed_sp_testing']))
#     mean_win_perc_dict['Fixed_Bar_testing'].\
#         append(np.mean(entry_walk_forward_dict['fixed_bar_testing']))
#     mean_win_perc_dict['Random_Exit_testing'].\
#         append(np.mean(entry_walk_forward_dict['random_exit_testing']))
#     mean_win_perc_dict['Not_Working'].\
#         append(100 * n_not_worked / n_total_cases)
    
#     win_pc_df = pd.DataFrame(mean_win_perc_dict)

#     return win_pc_df


In [7]:
def calculate_mean_win_perc_entry_testing(exec_dict, df):

    commission = exec_dict['COMMISSION']
    slippage = exec_dict['SLIPPAGE'] 
    init_inv = exec_dict['AVAILABLE_CAPITAL']
    trade_size = exec_dict['TRADE_SIZE'] 

    signal_idxs = list(exec_dict['buy_idxs'])
    signal_idxs.extend(list(exec_dict['sell_idxs']))
    signal_idxs = sorted(signal_idxs)
    signal_idxs_true = [i - 1 for i in signal_idxs]

    buy_idxs = [i - 1 for i in list(exec_dict['buy_idxs'])]
    df['buy'] = 0
    df.loc[df.index.isin(buy_idxs), 'buy'] = 1

    sell_idxs = [i - 1 for i in list(exec_dict['sell_idxs'])]
    df['sell'] = 0
    df.loc[df.index.isin(sell_idxs), 'sell'] = -1

    df['signal'] = df['buy'] + df['sell']

    df['new_signal'] = 0
    df.loc[df.index.isin(signal_idxs_true), 'new_signal'] = df.loc[df.index.isin(signal_idxs_true), 'signal'].values
    df['signal_prices'] = 0
    df.loc[df.index.isin(signal_idxs), 'signal_prices'] = df.loc[df.index.isin(signal_idxs), 'btc_open'].values

    signal_list = np.zeros((2, df.shape[0]))
    signal_list[0][1:] = df['new_signal'].values[:-1]
    signal_list[1] = df['signal_prices'].values

    # fixed stop and target exit testing
    exit_list = get_exit_entry_testing1(
        close_prices=df['btc_close'].values, 
        open_prices=df['btc_open'].values,  
        signal_list=signal_list,
        stoploss_th=50,
        takeprofit_th=100,
        commission=commission, 
        slippage=slippage, 
        init_inv=init_inv, 
        trade_size=trade_size
    )

    signal_list, exit_list = get_signals(signal_list, exit_list)
    pos_open_prices, pos_exit_prices = create_position_open_prices(signal_list, exit_list)

    pnl_list = get_pnl_testing(
        trade_close_prices=pos_exit_prices,
        signal_list=exit_list[0], 
        trade_open_prices=pos_open_prices,
        commission=commission, 
        slippage=slippage, 
        init_inv=init_inv, 
        trade_size=trade_size
    )

    fixed_winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)

    # fixed bar exit testing
    exit_list = get_exit_entry_testing2( 
        open_prices=df['btc_open'].values,  
        signal_list=signal_list,
        n_exit_bars=5
    )

    signal_list, exit_list = get_signals(signal_list, exit_list)
    pos_open_prices, pos_exit_prices = create_position_open_prices(signal_list, exit_list)

    pnl_list = get_pnl_testing(
        trade_close_prices=pos_exit_prices,
        signal_list=exit_list[0], 
        trade_open_prices=pos_open_prices,
        commission=commission, 
        slippage=slippage, 
        init_inv=init_inv, 
        trade_size=trade_size
    )

    fixed_bar_winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)

    # random exit testing
    exit_list = get_exit_entry_testing3( 
        open_prices=df['btc_open'].values,  
        signal_list=signal_list
    )

    signal_list, exit_list = get_signals(signal_list, exit_list)
    pos_open_prices, pos_exit_prices = create_position_open_prices(signal_list, exit_list)

    pnl_list = get_pnl_testing(
        trade_close_prices=pos_exit_prices,
        signal_list=exit_list[0], 
        trade_open_prices=pos_open_prices,
        commission=commission, 
        slippage=slippage, 
        init_inv=init_inv, 
        trade_size=trade_size
    )

    random_winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)

    return fixed_winning_percent, fixed_bar_winning_percent, random_winning_percent

def get_entry_win_pc_df(entry_walk_forward_dict, n_not_worked, n_total_cases):
    
    entry_mean_win_perc_dict = defaultdict(list)

    entry_mean_win_perc_dict['Fixed_StopLoss_TakeProfit_testing'].\
        append(np.mean(entry_walk_forward_dict['fixed_sp_testing']))
    entry_mean_win_perc_dict['Fixed_Bar_testing'].\
        append(np.mean(entry_walk_forward_dict['fixed_bar_testing']))
    entry_mean_win_perc_dict['Random_Exit_testing'].\
        append(np.mean(entry_walk_forward_dict['random_exit_testing']))
    entry_mean_win_perc_dict['Not_Working'].\
        append(100 * n_not_worked / n_total_cases)
        
    entry_win_pc_df = pd.DataFrame(entry_mean_win_perc_dict)

    return entry_win_pc_df


## Exit testing

### Similar approach entry

In [8]:
@njit(cache=True)
def get_entry_exit_testing1(
    close_prices,
    open_prices,
    n_bars
):

    signal_list = np.zeros((2, len(close_prices)))

    # n_bars = np.int64(n_bars)

    for i in range(n_bars, len(signal_list[0])):
            
        if close_prices[i - n_bars] - close_prices[i - 1] > 0:
            signal_list[0][i] = 1
            signal_list[1][i] = open_prices[i]
        elif close_prices[i - n_bars] - close_prices[i - 1] < 0:
           signal_list[0][i] = -1
           signal_list[1][i] = open_prices[i]
        else:
            pass
        
    return signal_list

@njit(cache=True)
def get_rsi(close_prices, prev_close_prices, length=14):
    # Create numpy arrays to store the gain/loss values
    gains = np.zeros(len(close_prices))
    losses = np.zeros(len(close_prices))

    # Iterate through the data frame and calculate the gain/loss for each period
    for i in range(1, len(close_prices)):
        change = close_prices[i] - prev_close_prices[i]
        if change > 0:
            gains[i] = change
        elif change < 0:
            losses[i] = abs(change)

    # Calculate the average gain and loss for each period
    avg_gains = np.zeros(len(close_prices))
    avg_losses = np.zeros(len(close_prices))
    for i in range(length, len(close_prices)):
        avg_gains[i] = np.mean(gains[i-length:i])
        avg_losses[i] = np.mean(losses[i-length:i])

    # Calculate the relative strength and RSI for each period
    rs = np.zeros(len(close_prices))
    rsi = np.zeros(len(close_prices))
    
    for i in range(len(close_prices)):
        if i+1 < length:
            rsi[i] = -999
        elif avg_losses[i] == 0:
            rs[i] = avg_gains[i]
            rsi[i] = 100
        else:
            rs[i] = avg_gains[i] / avg_losses[i]
            rsi[i] = 100 - (100 / (1 + rs[i]))

    return rsi

@njit(cache=True)
def get_entry_exit_testing2(
    close_prices, open_prices, prev_close_prices, rsi_window_size, rsi_threshold
):

    signal_list = np.zeros((2, len(close_prices)))

    rsi = get_rsi(close_prices, prev_close_prices, length=rsi_window_size)

    for i in range(len(close_prices)-1):

        if i < rsi_window_size - 1 or rsi[i] == -999:
            continue
       
        if rsi[i] < rsi_threshold:
            signal_list[0][i+1] = 1
            signal_list[1][i+1] = open_prices[i+1]
        elif rsi[i] > (100 - rsi_threshold):
            signal_list[0][i+1] = -1
            signal_list[1][i+1] = open_prices[i+1]
        else:
            pass

    return signal_list


### Random entry

In [9]:
@njit(cache=True)
def get_entry_exit_testing3(
    open_prices
):

    signal_list = np.zeros((2, len(open_prices)))

    for i in range(len(open_prices)-1):

        if np.random.rand() > 0.7:
            signal_list[0][i] = 1
            signal_list[1][i] = open_prices[i]
        elif np.random.rand() < 0.3:
            signal_list[0][i] = -1
            signal_list[1][i] = open_prices[i]
        else:
            pass

    return signal_list


### Winning percentage

In [10]:
# def calculate_mean_win_perc_exit_testing(data, text_code):

#     bars_per_5week = 7 * 60 * 24 * 5
#     n_bars_per_year = 7 * 60 * 24 * 52

#     if data.shape[0] < n_bars_per_year:
#         n_bars_per_year = data.shape[0]

#     n_not_worked = 0
#     n_total_cases = 0

#     exit_walk_forward_dict = defaultdict(list)

#     for idx in range(0, n_bars_per_year, bars_per_5week):

#         n_total_cases += 1

#         df = data.iloc[idx:idx+bars_per_5week, :]
#         df.reset_index(drop=True, inplace=True)

#         price_data = {}
#         for col in df.columns:
#             if col == 'datetime':
#                 continue
#             else:
#                 price_data[col] = df[col].values

#         try:

#             exec_dict = {'price_data': price_data}
#             exec(text_code, exec_dict)

#             commission = exec_dict['COMMISSION']
#             slippage = exec_dict['SLIPPAGE'] 
#             init_inv = exec_dict['AVAILABLE_CAPITAL']
#             trade_size = exec_dict['TRADE_SIZE']       

#             signal_idxs = list(exec_dict['buy_idxs'])
#             signal_idxs.extend(list(exec_dict['sell_idxs']))
#             signal_idxs = sorted(signal_idxs)
#             signal_idxs_true = [i - 1 for i in signal_idxs]

#             buy_idxs = [i - 1 for i in list(exec_dict['buy_idxs'])]
#             df['buy'] = 0
#             df.loc[df.index.isin(buy_idxs), 'buy'] = 1

#             sell_idxs = [i - 1 for i in list(exec_dict['sell_idxs'])]
#             df['sell'] = 0
#             df.loc[df.index.isin(sell_idxs), 'sell'] = -1

#             df['signal'] = df['buy'] + df['sell']

#             df['exit_signal'] = 0
#             df.loc[df.index.isin(signal_idxs_true[1:]), 'exit_signal'] = df.loc[df.index.isin(signal_idxs_true[1:]), 'signal'].values
#             df['exit_prices'] = 0
#             df.loc[df.index.isin(signal_idxs[1:]), 'exit_prices'] = df.loc[df.index.isin(signal_idxs[1:]), 'btc_open'].values

#             exit_list = np.zeros((2, df.shape[0]))
#             exit_list[0][1:] = df['exit_signal'].values[:-1]
#             exit_list[1] = df['exit_prices'].values

#             # replacing entry with trend following entry
#             signal_list = get_entry_exit_testing1(
#                 close_prices=df['btc_close'].values,
#                 open_prices=df['btc_open'].values,
#                 n_bars=5
#             )

#             signal_list, exit_list = get_signals(signal_list, exit_list)
#             pos_open_prices, pos_exit_prices = create_position_open_prices(signal_list, exit_list)

#             pnl_list = get_pnl_testing(
#                 trade_close_prices=pos_exit_prices,
#                 signal_list=exit_list[0], 
#                 trade_open_prices=pos_open_prices,
#                 commission=commission, 
#                 slippage=slippage, 
#                 init_inv=init_inv, 
#                 trade_size=trade_size
#             )

#             fixed_winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)
#             exit_walk_forward_dict['trend_entry_testing'].append(fixed_winning_percent)

#             # replacing entry with countertrend entry
#             signal_list = get_entry_exit_testing2(
#                 close_prices=df['btc_close'].values, 
#                 open_prices=df['btc_open'].values, 
#                 prev_close_prices=df['btc_close'].shift(1).fillna(method='bfill').values, 
#                 rsi_window_size=10, 
#                 rsi_threshold = 20
#             )

#             signal_list, exit_list = get_signals(signal_list, exit_list)
#             pos_open_prices, pos_exit_prices = create_position_open_prices(signal_list, exit_list)

#             pnl_list = get_pnl_testing(
#                 trade_close_prices=pos_exit_prices,
#                 signal_list=exit_list[0], 
#                 trade_open_prices=pos_open_prices,
#                 commission=commission, 
#                 slippage=slippage, 
#                 init_inv=init_inv, 
#                 trade_size=trade_size
#             )

#             fixed_bar_winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)
#             exit_walk_forward_dict['countertrend_entry_testing'].append(fixed_bar_winning_percent)

#             # random entry testing
#             signal_list = get_entry_exit_testing3(
#                 open_prices=df['btc_open'].values
#             )

#             signal_list, exit_list = get_signals(signal_list, exit_list)
#             pos_open_prices, pos_exit_prices = create_position_open_prices(signal_list, exit_list)

#             pnl_list = get_pnl_testing(
#                 trade_close_prices=pos_exit_prices,
#                 signal_list=exit_list[0], 
#                 trade_open_prices=pos_open_prices,
#                 commission=commission, 
#                 slippage=slippage, 
#                 init_inv=init_inv, 
#                 trade_size=trade_size
#             )

#             random_winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)
#             exit_walk_forward_dict['random_entry_testing'].append(random_winning_percent)

#         except:

#             n_not_worked += 1

#     mean_win_perc_dict = defaultdict(list)

#     mean_win_perc_dict['Trend_testing'].\
#         append(np.mean(exit_walk_forward_dict['trend_entry_testing']))
#     mean_win_perc_dict['Countertrend_testing'].\
#         append(np.mean(exit_walk_forward_dict['countertrend_entry_testing']))
#     mean_win_perc_dict['Random_Entry_testing'].\
#         append(np.mean(exit_walk_forward_dict['random_entry_testing']))
#     mean_win_perc_dict['Not_Working'].\
#         append(100 * n_not_worked / n_total_cases)
    
#     win_pc_df = pd.DataFrame(mean_win_perc_dict)

#     return win_pc_df


In [11]:
def calculate_mean_win_perc_exit_testing(exec_dict, df):

    commission = exec_dict['COMMISSION']
    slippage = exec_dict['SLIPPAGE'] 
    init_inv = exec_dict['AVAILABLE_CAPITAL']
    trade_size = exec_dict['TRADE_SIZE']       

    signal_idxs = list(exec_dict['buy_idxs'])
    signal_idxs.extend(list(exec_dict['sell_idxs']))
    signal_idxs = sorted(signal_idxs)
    signal_idxs_true = [i - 1 for i in signal_idxs]

    buy_idxs = [i - 1 for i in list(exec_dict['buy_idxs'])]
    df['buy'] = 0
    df.loc[df.index.isin(buy_idxs), 'buy'] = 1

    sell_idxs = [i - 1 for i in list(exec_dict['sell_idxs'])]
    df['sell'] = 0
    df.loc[df.index.isin(sell_idxs), 'sell'] = -1

    df['signal'] = df['buy'] + df['sell']

    df['exit_signal'] = 0
    df.loc[df.index.isin(signal_idxs_true[1:]), 'exit_signal'] = df.loc[df.index.isin(signal_idxs_true[1:]), 'signal'].values
    df['exit_prices'] = 0
    df.loc[df.index.isin(signal_idxs[1:]), 'exit_prices'] = df.loc[df.index.isin(signal_idxs[1:]), 'btc_open'].values

    exit_list = np.zeros((2, df.shape[0]))
    exit_list[0][1:] = df['exit_signal'].values[:-1]
    exit_list[1] = df['exit_prices'].values

    # replacing entry with trend following entry
    signal_list = get_entry_exit_testing1(
        close_prices=df['btc_close'].values,
        open_prices=df['btc_open'].values,
        n_bars=5
    )

    signal_list, exit_list = get_signals(signal_list, exit_list)
    pos_open_prices, pos_exit_prices = create_position_open_prices(signal_list, exit_list)

    pnl_list = get_pnl_testing(
        trade_close_prices=pos_exit_prices,
        signal_list=exit_list[0], 
        trade_open_prices=pos_open_prices,
        commission=commission, 
        slippage=slippage, 
        init_inv=init_inv, 
        trade_size=trade_size
    )

    trend_winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)

    # replacing entry with countertrend entry
    signal_list = get_entry_exit_testing2(
        close_prices=df['btc_close'].values, 
        open_prices=df['btc_open'].values, 
        prev_close_prices=df['btc_close'].shift(1).fillna(method='bfill').values, 
        rsi_window_size=10, 
        rsi_threshold = 20
    )

    signal_list, exit_list = get_signals(signal_list, exit_list)
    pos_open_prices, pos_exit_prices = create_position_open_prices(signal_list, exit_list)

    pnl_list = get_pnl_testing(
        trade_close_prices=pos_exit_prices,
        signal_list=exit_list[0], 
        trade_open_prices=pos_open_prices,
        commission=commission, 
        slippage=slippage, 
        init_inv=init_inv, 
        trade_size=trade_size
    )

    countertrend_winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)

    # random entry testing
    signal_list = get_entry_exit_testing3(
        open_prices=df['btc_open'].values
    )

    signal_list, exit_list = get_signals(signal_list, exit_list)
    pos_open_prices, pos_exit_prices = create_position_open_prices(signal_list, exit_list)

    pnl_list = get_pnl_testing(
        trade_close_prices=pos_exit_prices,
        signal_list=exit_list[0], 
        trade_open_prices=pos_open_prices,
        commission=commission, 
        slippage=slippage, 
        init_inv=init_inv, 
        trade_size=trade_size
    )

    random_winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)


    return trend_winning_percent, countertrend_winning_percent, random_winning_percent

def get_exit_win_pc_df(exit_walk_forward_dict, n_not_worked, n_total_cases):

    exit_mean_win_perc_dict = defaultdict(list)

    exit_mean_win_perc_dict['Trend_testing'].\
        append(np.mean(exit_walk_forward_dict['trend_entry_testing']))
    exit_mean_win_perc_dict['Countertrend_testing'].\
        append(np.mean(exit_walk_forward_dict['countertrend_entry_testing']))
    exit_mean_win_perc_dict['Random_Entry_testing'].\
        append(np.mean(exit_walk_forward_dict['random_entry_testing']))
    exit_mean_win_perc_dict['Not_Working'].\
        append(100 * n_not_worked / n_total_cases)

    exit_win_pc_df = pd.DataFrame(exit_mean_win_perc_dict)

    return exit_win_pc_df


## Core testing

### Winning percentage

In [12]:
# def calculate_mean_win_perc_core_testing(data, text_code):

#     bars_per_5week = 7 * 60 * 24 * 5
#     n_bars_per_year = 7 * 60 * 24 * 52

#     if data.shape[0] < n_bars_per_year:
#         n_bars_per_year = data.shape[0]

#     n_not_worked = 0
#     n_total_cases = 0

#     core_walk_forward_dict = defaultdict(list)

#     for idx in range(0, n_bars_per_year, bars_per_5week):

#         n_total_cases += 1

#         df = data.iloc[idx:idx+bars_per_5week, :]
#         df.reset_index(drop=True, inplace=True)

#         price_data = {}
#         for col in df.columns:
#             if col == 'datetime':
#                 continue
#             else:
#                 price_data[col] = df[col].values

#         try:
#             exec_dict = {'price_data': price_data}
#             exec(text_code, exec_dict)
#             pnl_list = exec_dict['all_arr']

#             winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)
#             core_walk_forward_dict['core_testing'].append(winning_percent)
#         except:
#             n_not_worked += 1

#     mean_win_perc_dict = defaultdict(list)
#     mean_win_perc_dict['Core_Testing'].\
#         append(np.mean(core_walk_forward_dict['core_testing']))
#     mean_win_perc_dict['Not_Working'].\
#         append(100 * n_not_worked / n_total_cases)
    
#     win_pc_df = pd.DataFrame(mean_win_perc_dict)

#     return win_pc_df


In [13]:
def calculate_mean_win_perc_core_testing(exec_dict):
            
    pnl_list = exec_dict['all_arr']

    winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)
    
    return winning_percent

def get_core_win_pc_df(core_walk_forward_dict, n_not_worked, n_total_cases):

    core_mean_win_perc_dict = defaultdict(list)
    core_mean_win_perc_dict['Core_Testing'].\
        append(np.mean(core_walk_forward_dict['core_testing']))
    core_mean_win_perc_dict['Not_Working'].\
        append(100 * n_not_worked / n_total_cases)

    core_win_pc_df = pd.DataFrame(core_mean_win_perc_dict)

    return core_win_pc_df


### Performance

In [14]:
# @njit(cache=True)
# def get_drawdown(pnl_list):

#     # running_total = np.cumsum(pnl_list)
#     # max_running_total = np.maximum.accumulate(running_total)

#     running_total = np.zeros(len(pnl_list))
#     max_running_total = np.zeros(len(pnl_list))

#     for i in range(len(pnl_list)):
#         if i == 0:
#             running_total[i] = pnl_list[i]
#             max_running_total[i] = pnl_list[i]
#         else:
#             running_total[i] = running_total[i-1] + pnl_list[i]
#             max_running_total[i] = max([max_running_total[i-1], pnl_list[i]])

#     min_total = running_total[np.argmin(running_total)]

#     max_total = max_running_total[np.argmin(running_total)]

#     if max_total == 0:
#         max_total = 0.00001

#     max_drawdown = 100 * (min_total - max_total) / max_total

#     if max_drawdown == -1:
#         max_drawdown = 0

#     return max_drawdown

@njit(cache=True)
def get_drawdown(pnl_list):
    max_dd = 0.0
    peak = pnl_list[0]

    for i in range(1, len(pnl_list)):
        if pnl_list[i] > peak:
            peak = pnl_list[i]
        drawdown = 100 * (peak - pnl_list[i]) / peak
        if drawdown > max_dd:
            max_dd = drawdown

    return max_dd

@njit(cache=True)
def get_drawdown_duration(pnl_arr):
    peak = pnl_arr[0]
    max_duration = 0
    current_duration = 0

    for pnl in pnl_arr:
        if pnl < peak:  # We are in a drawdown
            current_duration += 1
        else:  # We found a new peak
            peak = pnl
            current_duration = 0

        max_duration = max(max_duration, current_duration)

    return max_duration

@njit(cache=True)
def get_sharpe_ratio(
    pnl_list, 
    risk_free_rate=0
):
    # mean_return = np.mean(pnl_list)
    mean_return = 0
    for i in range(len(pnl_list)):
        mean_return += pnl_list[i]

    if len(pnl_list) == 0:
        mean_return = 0
    else:
        mean_return = mean_return / len(pnl_list)

    # std_dev = np.std(pnl_list)
    std_dev = 0
    for i in range(len(pnl_list)):
        std_dev += (pnl_list[i] - mean_return) ** 2

    if len(pnl_list) == 1:
        std_dev = np.sqrt(std_dev/len(pnl_list))
    else:
        std_dev = np.sqrt(std_dev/(len(pnl_list) - 1))

    if std_dev == 0:
        std_dev += 0.0001

    sharpe_ratio = (mean_return - risk_free_rate) / std_dev
    return sharpe_ratio

@njit(cache=True)
def get_sortino_ratio(profit_loss, target_pl=0):

    downside_pnl = profit_loss[profit_loss < target_pl]
    if len(downside_pnl) == 0:
        return 0

    downside_deviation = np.sqrt(np.mean((downside_pnl - target_pl) ** 2))
    if downside_deviation == 0:
        return 0

    mean_pnl = np.mean(profit_loss)
    sortino_ratio = (mean_pnl - target_pl) / downside_deviation

    return sortino_ratio



In [15]:
# def calculate_mean_performance(data, text_code, monkey_test=False):

#     bars_per_5week = 7 * 60 * 24 * 5
#     n_bars_per_year = 7 * 60 * 24 * 52

#     if data.shape[0] < n_bars_per_year:
#         n_bars_per_year = data.shape[0]

#     n_not_worked = 0
#     n_total_cases = 0

#     performance_walk_forward_dict = defaultdict(list)

#     for idx in range(0, n_bars_per_year, bars_per_5week):

#         n_total_cases += 1

#         df = data.iloc[idx:idx+bars_per_5week, :]
#         df.reset_index(drop=True, inplace=True)

#         price_data = {}
#         for col in df.columns:
#             if col == 'datetime':
#                 continue
#             else:
#                 price_data[col] = df[col].values

#         try:
#             exec_dict = {'price_data': price_data}
#             exec(text_code, exec_dict)
#             pnl_list = exec_dict['all_arr']

#             init_inv = exec_dict['AVAILABLE_CAPITAL']
#             trade_size = exec_dict['TRADE_SIZE']       

#             signal_idxs = list(exec_dict['buy_idxs'])
#             signal_idxs.extend(list(exec_dict['sell_idxs']))
#             signal_idxs = sorted(signal_idxs)

#             performance_walk_forward_dict['n_trades'].append(len(signal_idxs) - 1)
#             overall_pnl = np.sum(pnl_list)
#             performance_walk_forward_dict['pnl'].append(overall_pnl)
#             performance_walk_forward_dict['roi'].append(100 * overall_pnl / (trade_size * init_inv))
#             performance_walk_forward_dict['avg_drawdown'].append(exec_dict['avg_drawdown'])
#             max_dd = get_drawdown(pnl_list)
#             performance_walk_forward_dict['drawdown'].append(max_dd)
#             performance_walk_forward_dict['drawdown_dur'].append(get_drawdown_duration(pnl_list))
#             performance_walk_forward_dict['pnl_avgd_ratio'].append(exec_dict['fitness'])
#             annualized_sharpe_ration = np.sqrt(525600) * get_sharpe_ratio(pnl_list, risk_free_rate=0)
#             performance_walk_forward_dict['sharpe_ratio'].append(annualized_sharpe_ration)
#             annualized_sortino_ratio = np.sqrt(525600) * get_sortino_ratio(pnl_list)
#             performance_walk_forward_dict['sortino_ratio'].append(annualized_sortino_ratio)

#             if monkey_test:
#                 pnl_mren_arr = exec_dict['pnl_mren_arr']
#                 max_dd_mren_arr = exec_dict['max_dd_mren_arr']

#                 pnl_mren_good_cases = np.sum(np.where(overall_pnl > pnl_mren_arr, 1, 0))
#                 max_dd_mren_good_cases = np.sum(np.where(max_dd < max_dd_mren_arr, 1, 0))
#                 performance_walk_forward_dict['mt_pnl'].append(100 * pnl_mren_good_cases / len(pnl_mren_arr))
#                 performance_walk_forward_dict['mt_mdd'].append(100 * max_dd_mren_good_cases / len(max_dd_mren_arr))

#         except:
#             n_not_worked += 1

#     mean_perf_dict = defaultdict(list)
#     mean_perf_dict['N_Trades'].\
#         append(np.mean(performance_walk_forward_dict['n_trades']))
#     mean_perf_dict['PNL'].\
#         append(np.mean(performance_walk_forward_dict['pnl']))
#     mean_perf_dict['ROI (%)'].\
#         append(np.mean(performance_walk_forward_dict['roi']))
#     mean_perf_dict['AVG_Drawdown'].\
#         append(np.mean(performance_walk_forward_dict['avg_drawdown']))
#     mean_perf_dict['Drawdown (%)'].\
#         append(np.mean(performance_walk_forward_dict['drawdown']))
#     mean_perf_dict['Drawdown_Duration'].\
#         append(np.mean(performance_walk_forward_dict['drawdown_dur']))
#     mean_perf_dict['PNL_AVGD_Ratio'].\
#         append(np.mean(performance_walk_forward_dict['pnl_avgd_ratio']))
#     mean_perf_dict['Sharpe_Ratio'].\
#         append(np.mean(performance_walk_forward_dict['sharpe_ratio']))
#     mean_perf_dict['Sortino_Ratio'].\
#         append(np.mean(performance_walk_forward_dict['sortino_ratio']))
#     mean_perf_dict['Not_Working'].\
#         append(100 * n_not_worked / n_total_cases)
    
#     if monkey_test:
#         mean_perf_dict['MT_PNL_D'].\
#             append(np.mean(performance_walk_forward_dict['mt_pnl']))
#         mean_perf_dict['MT_MDD_D'].\
#             append(np.mean(performance_walk_forward_dict['mt_mdd']))
#         # mean_perf_dict['MT_PNL_D_Min'].\
#         #     append(np.min(performance_walk_forward_dict['mt_pnl']))
#         # mean_perf_dict['MT_MDD_D_Min'].\
#         #     append(np.min(performance_walk_forward_dict['mt_mdd']))
#         # mean_perf_dict['MT_PNL_D_Max'].\
#         #     append(np.max(performance_walk_forward_dict['mt_pnl']))
#         # mean_perf_dict['MT_MDD_D_Max'].\
#         #     append(np.max(performance_walk_forward_dict['mt_mdd']))
    
#     perf_df = pd.DataFrame(mean_perf_dict)

#     return perf_df


In [16]:
def calculate_mean_performance(exec_dict, monkey_test=False):

    pnl_list = exec_dict['all_arr']

    init_inv = exec_dict['AVAILABLE_CAPITAL']
    trade_size = exec_dict['TRADE_SIZE']       

    signal_idxs = list(exec_dict['buy_idxs'])
    signal_idxs.extend(list(exec_dict['sell_idxs']))
    signal_idxs = sorted(signal_idxs)

    metric_dict = {}

    metric_dict['n_trades'] = len(signal_idxs) - 1
    metric_dict['overall_pnl'] = np.sum(pnl_list)
    metric_dict['roi'] = 100 * metric_dict['overall_pnl'] / (trade_size * init_inv)
    metric_dict['avg_drawdown'] = exec_dict['avg_drawdown']
    metric_dict['max_dd'] = get_drawdown(pnl_list)
    metric_dict['drawdown_dur'] = get_drawdown_duration(pnl_list)
    metric_dict['pnl_avgd_ratio'] = exec_dict['fitness']

    annualized_sharpe_ration = np.sqrt(525600) * get_sharpe_ratio(pnl_list, risk_free_rate=0)
    metric_dict['sharpe_ratio'] = annualized_sharpe_ration

    annualized_sortino_ratio = np.sqrt(525600) * get_sortino_ratio(pnl_list)
    metric_dict['sortino_ratio'] = annualized_sortino_ratio

    if monkey_test:

        pnl_mren_arr = exec_dict['pnl_mren_arr']
        max_dd_mren_arr = exec_dict['max_dd_mren_arr']

        pnl_mren_good_cases = np.sum(np.where(metric_dict['overall_pnl'] > pnl_mren_arr, 1, 0))
        metric_dict['mt_pnl'] = 100 * pnl_mren_good_cases / len(pnl_mren_arr)

        max_dd_mren_good_cases = np.sum(np.where(metric_dict['max_dd'] < max_dd_mren_arr, 1, 0))
        metric_dict['mt_mdd'] = 100 * max_dd_mren_good_cases / len(max_dd_mren_arr)

    return metric_dict

def get_perf_df(performance_walk_forward_dict, n_not_worked, n_total_cases):

    mean_perf_dict = defaultdict(list)
    
    mean_perf_dict['N_Trades'].\
        append(np.mean(performance_walk_forward_dict['n_trades']))
    mean_perf_dict['PNL'].\
        append(np.mean(performance_walk_forward_dict['pnl']))
    mean_perf_dict['ROI (%)'].\
        append(np.mean(performance_walk_forward_dict['roi']))
    mean_perf_dict['AVG_Drawdown'].\
        append(np.mean(performance_walk_forward_dict['avg_drawdown']))
    mean_perf_dict['Drawdown (%)'].\
        append(np.mean(performance_walk_forward_dict['drawdown']))
    mean_perf_dict['Drawdown_Duration'].\
        append(np.mean(performance_walk_forward_dict['drawdown_dur']))
    mean_perf_dict['PNL_AVGD_Ratio'].\
        append(np.mean(performance_walk_forward_dict['pnl_avgd_ratio']))
    mean_perf_dict['Sharpe_Ratio'].\
        append(np.mean(performance_walk_forward_dict['sharpe_ratio']))
    mean_perf_dict['Sortino_Ratio'].\
        append(np.mean(performance_walk_forward_dict['sortino_ratio']))
    mean_perf_dict['Not_Working'].\
        append(100 * n_not_worked / n_total_cases)
    
    if 'mt_pnl' in performance_walk_forward_dict.keys():
        mean_perf_dict['MT_PNL_D'].\
            append(np.mean(performance_walk_forward_dict['mt_pnl']))
        mean_perf_dict['MT_MDD_D'].\
            append(np.mean(performance_walk_forward_dict['mt_mdd']))

    perf_df = pd.DataFrame(mean_perf_dict)

    return perf_df


## Equity curve Monte Carlo simulation

In [17]:
@njit(cache=True)
def get_random_idxs4mc(arr, num_elements):
    # Get the length of the input array
    n = len(arr)

    if num_elements > n:
        raise ValueError("num_elements must be less than or equal to the length of the array.")

    # Generate random indices with replacement
    indices = np.random.randint(0, n, num_elements)

    # Return sorted indices
    return indices

@njit(cache=True)
def run_simulation(pnl_list, n_runs, init_inv, trade_size):

    max_dd_array = np.zeros(n_runs)
    dd_dur_array = np.zeros(n_runs)
    profit_array = np.zeros(n_runs)
    roi_array = np.zeros(n_runs)
    binary_profit_array = np.zeros(n_runs, dtype=np.int32)
    
    for i in range(n_runs):
        
        # Generate bootstrap sample
        temp_random_idxs = get_random_idxs4mc(arr=pnl_list, num_elements=len(pnl_list))
        temp_pnl_list = np.zeros(len(temp_random_idxs))
        for j in range(len(temp_random_idxs)):
            temp_pnl_list[j] = pnl_list[temp_random_idxs[j]]

        equity_curve_list = np.cumsum(temp_pnl_list)
        
        # Max Drawdown
        max_dd = get_drawdown(equity_curve_list)
        max_dd_array[i] = max_dd
        
        # Drawdown Duration
        dd_dur = get_drawdown_duration(equity_curve_list)
        dd_dur_array[i] = dd_dur
        
        # Profit
        profit = np.sum(temp_pnl_list)
        profit_array[i] = profit
        
        # ROI
        roi = 100 * profit / (init_inv * trade_size)
        roi_array[i] = roi
        
        # Binary Profit
        if profit > 0:
            binary_profit_array[i] = 1

    return max_dd_array, dd_dur_array, profit_array, roi_array, binary_profit_array


In [18]:
def get_mc_results(pnl_list, init_inv, trade_size, n_runs):

    max_dd_array, dd_dur_array, profit_array, roi_array, binary_profit_array = run_simulation(pnl_list, n_runs, init_inv, trade_size)

    mc_dict = {}
    mc_dict['median_max_dd'] = np.median(max_dd_array)
    mc_dict['median_dd_dur'] = np.median(dd_dur_array)
    mc_dict['median_profit'] = np.median(profit_array)
    mc_dict['median_return'] = np.median(roi_array)
    mc_dict['return_dd_ratio'] = np.median(roi_array) / np.median(max_dd_array)
    mc_dict['prob_profit'] = np.sum(binary_profit_array) / len(binary_profit_array)

    return mc_dict

In [19]:
# def get_mc_results(pnl_list, init_inv, trade_size, n_runs):

#     max_dd_list = []
#     dd_dur_list = []
#     profit_list = []
#     roi_list = []
#     binary_profit_list = []

#     for _ in range(n_runs):

#         temp_pnl_list = np.random.choice(pnl_list, size=len(pnl_list), replace=True)

#         max_dd = get_drawdown(temp_pnl_list)
#         max_dd_list.append(max_dd)

#         dd_dur = get_drawdown_duration(temp_pnl_list)
#         dd_dur_list.append(dd_dur)

#         profit_list.append(np.sum(temp_pnl_list))
#         roi_list.append(100 * np.sum(temp_pnl_list) / (init_inv * trade_size))
#         is_profit = 1 if np.sum(temp_pnl_list) > 0 else 0
#         binary_profit_list.append(is_profit)

#     mc_dict = {}
#     mc_dict['median_max_dd'] = np.median(max_dd_list)
#     mc_dict['median_dd_dur'] = np.median(dd_dur_list)
#     mc_dict['median_profit'] = np.median(profit_list)
#     mc_dict['median_return'] = np.median(roi_list)
#     mc_dict['return_dd_ratio'] = np.median(roi_list) / np.median(max_dd_list)
#     mc_dict['prob_profit'] = np.sum(binary_profit_list) / len(binary_profit_list)

#     return mc_dict

In [20]:
# def calculate_mc_performance(data, text_code):

#     bars_per_5week = 7 * 60 * 24 * 5
#     n_bars_per_year = 7 * 60 * 24 * 52

#     if data.shape[0] < n_bars_per_year:
#         n_bars_per_year = data.shape[0]

#     n_not_worked = 0
#     n_total_cases = 0

#     performance_walk_forward_dict = defaultdict(list)

#     for idx in range(0, n_bars_per_year, bars_per_5week):

#         n_total_cases += 1

#         df = data.iloc[idx:idx+bars_per_5week, :]
#         df.reset_index(drop=True, inplace=True)

#         price_data = {}
#         for col in df.columns:
#             if col == 'datetime':
#                 continue
#             else:
#                 price_data[col] = df[col].values

#         try:
#             exec_dict = {'price_data': price_data}
#             exec(text_code, exec_dict)
#             pnl_list = exec_dict['all_arr']

#             init_inv = exec_dict['AVAILABLE_CAPITAL']
#             trade_size = exec_dict['TRADE_SIZE']       

#             mc_dict = get_mc_results(pnl_list, init_inv, trade_size, n_runs=10000)

#             performance_walk_forward_dict['median_max_dd'].append(mc_dict['median_max_dd'])
#             performance_walk_forward_dict['median_dd_dur'].append(mc_dict['median_dd_dur'])
#             performance_walk_forward_dict['median_profit'].append(mc_dict['median_profit'])
#             performance_walk_forward_dict['median_return'].append(mc_dict['median_return'])
#             performance_walk_forward_dict['return_dd_ratio'].append(mc_dict['return_dd_ratio'])
#             performance_walk_forward_dict['prob_profit'].append(mc_dict['prob_profit'])
#         except:
#             n_not_worked += 1

#     mean_perf_dict = defaultdict(list)
#     mean_perf_dict['median_drawdown (%)'].\
#         append(np.mean(performance_walk_forward_dict['median_max_dd']))
#     mean_perf_dict['median_drawdown_duration'].\
#         append(np.mean(performance_walk_forward_dict['median_dd_dur']))
#     mean_perf_dict['median_profit'].\
#         append(np.mean(performance_walk_forward_dict['median_profit']))
#     mean_perf_dict['median_ROI (%)'].\
#         append(np.mean(performance_walk_forward_dict['median_return']))
#     mean_perf_dict['ratio'].\
#         append(np.mean(performance_walk_forward_dict['return_dd_ratio']))
#     mean_perf_dict['prob'].\
#         append(np.mean(performance_walk_forward_dict['prob_profit']))
    
#     perf_df = pd.DataFrame(mean_perf_dict)

#     return perf_df


In [21]:
def calculate_mc_performance(exec_dict):
            
    pnl_list = exec_dict['all_arr']

    init_inv = exec_dict['AVAILABLE_CAPITAL']
    trade_size = exec_dict['TRADE_SIZE']       

    mc_dict = get_mc_results(pnl_list, init_inv, trade_size, n_runs=10000)

    return mc_dict

def get_mc_df(mc_walk_forward_dict, n_not_worked, n_total_cases):

    mean_mc_dict = defaultdict(list)

    mean_mc_dict['median_drawdown (%)'].\
        append(np.mean(mc_walk_forward_dict['median_max_dd']))
    mean_mc_dict['median_drawdown_duration'].\
        append(np.mean(mc_walk_forward_dict['median_dd_dur']))
    mean_mc_dict['median_profit'].\
        append(np.mean(mc_walk_forward_dict['median_profit']))
    mean_mc_dict['median_ROI (%)'].\
        append(np.mean(mc_walk_forward_dict['median_return']))
    mean_mc_dict['ratio'].\
        append(np.mean(mc_walk_forward_dict['return_dd_ratio']))
    mean_mc_dict['prob'].\
        append(np.mean(mc_walk_forward_dict['prob_profit']))
    mean_mc_dict['Not_Working'].\
        append(100 * n_not_worked / n_total_cases)

    mc_df = pd.DataFrame(mean_mc_dict)

    return mc_df


# Data loading function

In [22]:
def generate_52w_data(data_path):
    df = pd.read_csv(data_path)
    df['datetime'] = pd.to_datetime(df['datetime'])
    df = df.iloc[-(7 * 60 * 24 * 52):]
    df.sort_values('datetime', ascending=True, inplace=True)
    df.reset_index(inplace=True, drop=True)

    return df

# Testing strategies constructed from combination with other instruments

## Loading price and volume data

In [23]:
data_path = Path(r'C:/\Users/\vchar/\OneDrive/\Desktop/\ML Projects/\Upwork/\AlgoT_ML_Dev/\GrammarEvolution/\PonyGE2/\datasets/\all_data_1min.csv')
df_52w = generate_52w_data(data_path)
df_52w.head()

,datetime,6e_open,6e_high,6e_low,6e_close,6e_volume,aapl_open,aapl_high,aapl_low,aapl_close,...,zf_open,zf_high,zf_low,zf_close,zf_volume,zn_open,zn_high,zn_low,zn_close,zn_volume
0,2023-11-01 18:56:00,1.05600,1.05635,1.05590,1.05590,607,172.77,172.97,172.765,172.95,...,104.914062,104.945312,104.914062,104.937500,3727,106.812500,106.859375,106.796875,106.843750,11709
1,2023-11-01 18:58:00,1.05625,1.05640,1.05615,1.05630,396,173.10,173.27,173.060,173.25,...,104.937500,104.945312,104.921875,104.937500,3415,106.843750,106.859375,106.828125,106.859375,8757
2,2023-11-01 18:59:00,1.05630,1.05650,1.05605,1.05615,1569,173.24,173.24,173.090,173.19,...,104.937500,104.960938,104.929688,104.960938,13873,106.859375,106.875000,106.828125,106.875000,33414
3,2023-11-01 19:02:00,1.05650,1.05670,1.05640,1.05660,326,173.59,173.75,173.550,173.74,...,104.976562,105.015625,104.968750,105.007812,5523,106.906250,106.937500,106.890625,106.921875,7998
4,2023-11-01 19:03:00,1.05655,1.05675,1.05645,1.05665,327,173.72,173.73,173.650,173.72,...,105.000000,105.023438,105.000000,105.000000,4571,106.921875,106.953125,106.906250,106.906250,9335


## Loading strategies generated

In [24]:
strategy_file_path = Path(r'C:/\Users/\vchar/\OneDrive/\Desktop/\ML Projects/\Upwork/\AlgoT_ML_Dev/\GrammarEvolution/\PonyGE2/\derived_strategies/\ge_results_inst_ind_comb_numba.csv')
strategy_file_path = Path(r"C:/\Users/\vchar/\Downloads/\ge_results (2).csv")

try:
    df_str = pd.read_csv(strategy_file_path)
except:
    df_str = pd.read_csv(strategy_file_path, sep=';')
    
# df_str = df_str[(df_str['fitness'] < 0) & (df_str['fitness'] < -3) & (df_str['fitness'] > -50)]
df_str = df_str[(df_str['fitness'] < 0) & (df_str['fitness'] <= -1.5)]
df_str = df_str[~df_str.duplicated()]
df_str.sort_values('fitness', ascending=True, inplace=True)
df_str.reset_index(drop=True, inplace=True)
df_str

,buy,sell,fitness
0,"(get_lag(price_data['xrp_volume'], lag=2)[MAX_...","((get_lag(price_data['ng_volume'], lag=5)[MAX_...",-36068.239247
1,"(get_lag(price_data['xrp_volume'], lag=2)[MAX_...","((get_lag(price_data['ng_volume'], lag=5)[MAX_...",-36068.239247
2,"(get_lag(price_data['xrp_volume'], lag=2)[MAX_...","((get_lag(price_data['ng_volume'], lag=5)[MAX_...",-36068.239247
3,"(get_lag(price_data['xrp_volume'], lag=2)[MAX_...","((get_lag(price_data['ng_volume'], lag=5)[MAX_...",-36068.239247
4,"(get_lag(price_data['xrp_volume'], lag=2)[MAX_...","((get_lag(price_data['ng_volume'], lag=5)[MAX_...",-36068.239247
...,...,...,...
19527,"(np.cos(get_lag(price_data['tia_open'], lag=2)...","(np.sqrt(get_lag(price_data['nq_volume'], lag=...",-1.516622
19528,"get_lag(price_data['nvda_volume'], lag=2)[MAX_...","(np.sqrt(get_lag(price_data['amzn_high'], lag=...",-1.514053
19529,"(get_lag(price_data['coin_volume'], lag=3)[MAX...","np.cos(get_lag(price_data['gc_high'], lag=4)[M...",-1.509539
19530,np.cos(price_data['6e_low'][MAX_LAG:]) >= np.c...,(np.cos(price_data['tsla_open'][MAX_LAG:]) * n...,-1.507837


## Running the tests over the strategies

In [25]:
# final_entry_win_pc_df = pd.DataFrame()
# final_exit_win_pc_df = pd.DataFrame()
# final_core_win_pc_df = pd.DataFrame()
# final_perf_df = pd.DataFrame()
# final_mc_df = pd.DataFrame()

# lag_txt = '{i}'

# strategy_idx = 1

# for row in tqdm(df_str.iloc[:3000].itertuples()):#df_str.iloc[:10].itertuples()

#     buy_signal_txt = row.buy
#     sell_signal_txt = row.sell

#     txt_code = f'''import os
# CUR_DIR = os.getcwd()
# os.chdir('src')
# #import pandas as pd
# import numpy as np
# import gc
# from fitness.indicators import numba_indicators
# from fitness.performance.helper_func import merge_pnl, get_drawdowns, get_pnl, trading_signals, get_lag, get_monkey_test_results
# os.chdir(CUR_DIR)
# #from numba import njit
# COMMISSION = 0.015
# SLIPPAGE = 0.00005
# AVAILABLE_CAPITAL = 700000
# TRADE_SIZE = 0.5
# MAX_LAG = 5
# buy_idxs, sell_idxs = trading_signals(buy_signal={buy_signal_txt}, sell_signal={sell_signal_txt})
# if len(buy_idxs) == 0 or len(sell_idxs) == 0:
#     fitness = -999
# else:
#     buy_idxs = np.array(buy_idxs)
#     sell_idxs = np.array(sell_idxs)
#     open_prices = price_data['btc_open']
#     pnl_mren_arr, max_dd_mren_arr = get_monkey_test_results(open_prices, buy_idxs, sell_idxs, COMMISSION, SLIPPAGE, AVAILABLE_CAPITAL, TRADE_SIZE)
#     buy_prices = open_prices[np.isin(np.arange(len(open_prices)), buy_idxs)]
#     sell_prices = open_prices[np.isin(np.arange(len(open_prices)), sell_idxs)]
#     if buy_idxs[0] < sell_idxs[0]:
#         buy_arr = get_pnl(sell_prices, buy_prices, COMMISSION, SLIPPAGE, AVAILABLE_CAPITAL, TRADE_SIZE, 1)
#         buy_pnl = np.sum(buy_arr)
#         sell_arr = get_pnl(buy_prices[1:], sell_prices[:-1], COMMISSION, SLIPPAGE, AVAILABLE_CAPITAL, TRADE_SIZE, 0)
#         sell_pnl = np.sum(sell_arr)
#         all_arr = merge_pnl(buy_arr, sell_arr)
#     else:
#         sell_arr = get_pnl(buy_prices, sell_prices, COMMISSION, SLIPPAGE, AVAILABLE_CAPITAL, TRADE_SIZE, 0)
#         sell_pnl = np.sum(sell_arr)
#         buy_arr = get_pnl(sell_prices[1:], buy_prices[:-1], COMMISSION, SLIPPAGE, AVAILABLE_CAPITAL, TRADE_SIZE, 1)
#         buy_pnl = np.sum(buy_arr)
#         all_arr = merge_pnl(sell_arr, buy_arr)
#     total_pnl = buy_pnl + sell_pnl
#     equity_curve_arr = np.cumsum(all_arr)
#     drawdowns = get_drawdowns(equity_curve_arr)
#     avg_drawdown = np.sum(drawdowns[drawdowns!=0]) / len(drawdowns[drawdowns!=0])
#     fitness = total_pnl / avg_drawdown
#     if np.isnan(fitness) or total_pnl <= 0 or len(drawdowns[drawdowns!=0]) == 0:
#         fitness = -999
# gc.collect()'''

#     entry_win_pc_df = calculate_mean_win_perc_entry_testing(
#         data=df_52w, 
#         text_code=txt_code
#     )

#     temp_signal_df = pd.DataFrame({'strategy': f'strategy{strategy_idx}', 'buy': [buy_signal_txt], 'sell': [sell_signal_txt]})
#     entry_win_pc_df = pd.concat([temp_signal_df, entry_win_pc_df], axis=1)
#     final_entry_win_pc_df = pd.concat([final_entry_win_pc_df, entry_win_pc_df])

#     exit_win_pc_df = calculate_mean_win_perc_exit_testing(
#         data=df_52w, 
#         text_code=txt_code
#     )

#     exit_win_pc_df = pd.concat([temp_signal_df, exit_win_pc_df], axis=1)
#     final_exit_win_pc_df = pd.concat([final_exit_win_pc_df, exit_win_pc_df])

#     core_win_pc_df = calculate_mean_win_perc_core_testing(
#         data=df_52w, 
#         text_code=txt_code
#     )

#     core_win_pc_df = pd.concat([temp_signal_df, core_win_pc_df], axis=1)
#     final_core_win_pc_df = pd.concat([final_core_win_pc_df, core_win_pc_df])

#     perf_df = calculate_mean_performance(
#         data=df_52w, 
#         text_code=txt_code,
#         monkey_test=False
#     )

#     perf_df = pd.concat([temp_signal_df, perf_df], axis=1)
#     final_perf_df = pd.concat([final_perf_df, perf_df])

#     # mc_df = calculate_mc_performance(
#     #     data=df_52w, 
#     #     text_code=txt_code
#     # )

#     # mc_df = pd.concat([temp_signal_df, mc_df], axis=1)
#     # final_mc_df = pd.concat([final_mc_df, mc_df])

#     strategy_idx += 1

In [25]:
final_entry_win_pc_df = pd.DataFrame()
final_exit_win_pc_df = pd.DataFrame()
final_core_win_pc_df = pd.DataFrame()
final_perf_df = pd.DataFrame()
final_mc_df = pd.DataFrame()

equity_curve_dict = {}

bars_per_5week = 7 * 60 * 24 * 5
n_bars_per_year = 7 * 60 * 24 * 52

if df_52w.shape[0] < n_bars_per_year:
    n_bars_per_year = df_52w.shape[0]

lag_txt = '{i}'

strategy_idx = 1

for row in tqdm(df_str.itertuples()):#df_str.iloc[:10].itertuples()

    buy_signal_txt = row.buy
    sell_signal_txt = row.sell

    text_code = f'''import os
CUR_DIR = os.getcwd()
os.chdir('src')
#import pandas as pd
import numpy as np
import gc
from fitness.indicators import numba_indicators
from fitness.performance.helper_func import merge_pnl, get_drawdowns, get_pnl, trading_signals, get_lag, get_monkey_test_results
os.chdir(CUR_DIR)
#from numba import njit
COMMISSION = 0.015
SLIPPAGE = 0.00005
AVAILABLE_CAPITAL = 700000
TRADE_SIZE = 0.5
MAX_LAG = 5
buy_idxs, sell_idxs = trading_signals(buy_signal={buy_signal_txt}, sell_signal={sell_signal_txt})
if len(buy_idxs) == 0 or len(sell_idxs) == 0:
    fitness = -9999999
    avg_drawdown = -9999999
else:
    buy_idxs = np.array(buy_idxs)
    sell_idxs = np.array(sell_idxs)
    open_prices = price_data['btc_open']
    pnl_mren_arr, max_dd_mren_arr = get_monkey_test_results(open_prices, buy_idxs, sell_idxs, COMMISSION, SLIPPAGE, AVAILABLE_CAPITAL, TRADE_SIZE)
    buy_prices = open_prices[np.isin(np.arange(len(open_prices)), buy_idxs)]
    sell_prices = open_prices[np.isin(np.arange(len(open_prices)), sell_idxs)]
    if buy_idxs[0] < sell_idxs[0]:
        buy_arr = get_pnl(sell_prices, buy_prices, COMMISSION, SLIPPAGE, AVAILABLE_CAPITAL, TRADE_SIZE, 1)
        buy_pnl = np.sum(buy_arr)
        sell_arr = get_pnl(buy_prices[1:], sell_prices[:-1], COMMISSION, SLIPPAGE, AVAILABLE_CAPITAL, TRADE_SIZE, 0)
        sell_pnl = np.sum(sell_arr)
        all_arr = merge_pnl(buy_arr, sell_arr)
    else:
        sell_arr = get_pnl(buy_prices, sell_prices, COMMISSION, SLIPPAGE, AVAILABLE_CAPITAL, TRADE_SIZE, 0)
        sell_pnl = np.sum(sell_arr)
        buy_arr = get_pnl(sell_prices[1:], buy_prices[:-1], COMMISSION, SLIPPAGE, AVAILABLE_CAPITAL, TRADE_SIZE, 1)
        buy_pnl = np.sum(buy_arr)
        all_arr = merge_pnl(sell_arr, buy_arr)
    total_pnl = buy_pnl + sell_pnl
    equity_curve_arr = np.cumsum(all_arr)
    drawdowns = get_drawdowns(equity_curve_arr)
    if len(drawdowns[drawdowns!=0]) != 0:
        avg_drawdown = np.sum(drawdowns[drawdowns!=0]) / len(drawdowns[drawdowns!=0])
        fitness = total_pnl / avg_drawdown
    elif np.isnan(fitness) or total_pnl <= 0 or len(drawdowns[drawdowns!=0]) == 0:
        fitness = -9999999
        avg_drawdown = -9999999
gc.collect()'''
    
    entry_test_n_not_worked = 0
    exit_test_n_not_worked = 0
    core_test_n_not_worked = 0
    perf_n_not_worked = 0
    mc_n_not_worked = 0
    n_total_cases = 0

    equity_curve_dict = defaultdict(list)

    entry_walk_forward_dict = defaultdict(list)

    exit_walk_forward_dict = defaultdict(list)

    core_walk_forward_dict = defaultdict(list)

    performance_walk_forward_dict = defaultdict(list)

    mc_walk_forward_dict = defaultdict(list)

    for idx in range(0, n_bars_per_year, bars_per_5week):

        n_total_cases += 1

        df = df_52w.iloc[idx:idx+bars_per_5week, :]
        df.reset_index(drop=True, inplace=True)

        price_data = {}
        for col in df.columns:
            if col == 'datetime':
                continue
            else:
                price_data[col] = df[col].values

        exec_dict = {'price_data': price_data}
        exec(text_code, exec_dict)

        equity_curve_arr = exec_dict['equity_curve_arr']
        equity_curve_dict[strategy_idx].append(equity_curve_arr)

        try:
            fixed_winning_percent, fixed_bar_winning_percent, random_winning_percent = calculate_mean_win_perc_entry_testing(exec_dict, df)
            entry_walk_forward_dict['fixed_sp_testing'].append(fixed_winning_percent)
            entry_walk_forward_dict['fixed_bar_testing'].append(fixed_bar_winning_percent)
            entry_walk_forward_dict['random_exit_testing'].append(random_winning_percent)
        except:
            entry_test_n_not_worked += 1

        try:
            trend_winning_percent, countertrend_winning_percent, random_winning_percent = calculate_mean_win_perc_exit_testing(exec_dict, df)
            exit_walk_forward_dict['trend_entry_testing'].append(trend_winning_percent)
            exit_walk_forward_dict['countertrend_entry_testing'].append(countertrend_winning_percent)
            exit_walk_forward_dict['random_entry_testing'].append(random_winning_percent)
        except:
            exit_test_n_not_worked += 1

        try:
            winning_percent = calculate_mean_win_perc_core_testing(exec_dict)
            core_walk_forward_dict['core_testing'].append(winning_percent)
        except:
            core_test_n_not_worked += 1

        try:
            metric_dict = calculate_mean_performance(exec_dict, monkey_test=True)
            performance_walk_forward_dict['n_trades'].append(metric_dict['n_trades'])
            performance_walk_forward_dict['pnl'].append(metric_dict['overall_pnl'])
            performance_walk_forward_dict['roi'].append(metric_dict['roi'])
            performance_walk_forward_dict['avg_drawdown'].append(metric_dict['avg_drawdown'])
            performance_walk_forward_dict['drawdown'].append(metric_dict['max_dd'])
            performance_walk_forward_dict['drawdown_dur'].append(metric_dict['drawdown_dur'])
            performance_walk_forward_dict['pnl_avgd_ratio'].append(metric_dict['pnl_avgd_ratio'])
            performance_walk_forward_dict['sharpe_ratio'].append(metric_dict['sharpe_ratio'])
            performance_walk_forward_dict['sortino_ratio'].append(metric_dict['sortino_ratio'])
            if 'mt_pnl' in metric_dict.keys():
                performance_walk_forward_dict['mt_pnl'].append(metric_dict['mt_pnl'])
                performance_walk_forward_dict['mt_mdd'].append(metric_dict['mt_mdd'])
        except:
            perf_n_not_worked += 1

        try:
            mc_dict = calculate_mc_performance(exec_dict)
            mc_walk_forward_dict['median_max_dd'].append(mc_dict['median_max_dd'])
            mc_walk_forward_dict['median_dd_dur'].append(mc_dict['median_dd_dur'])
            mc_walk_forward_dict['median_profit'].append(mc_dict['median_profit'])
            mc_walk_forward_dict['median_return'].append(mc_dict['median_return'])
            mc_walk_forward_dict['return_dd_ratio'].append(mc_dict['return_dd_ratio'])
            mc_walk_forward_dict['prob_profit'].append(mc_dict['prob_profit'])
        except:
            mc_n_not_worked += 1

    temp_signal_df = pd.DataFrame({'strategy': f'strategy{strategy_idx}', 'buy': [buy_signal_txt], 'sell': [sell_signal_txt]})

    entry_win_pc_df = get_entry_win_pc_df(entry_walk_forward_dict, entry_test_n_not_worked, n_total_cases)
    entry_win_pc_df = pd.concat([temp_signal_df, entry_win_pc_df], axis=1)
    final_entry_win_pc_df = pd.concat([final_entry_win_pc_df, entry_win_pc_df])

    exit_win_pc_df = get_exit_win_pc_df(exit_walk_forward_dict, exit_test_n_not_worked, n_total_cases)
    exit_win_pc_df = pd.concat([temp_signal_df, exit_win_pc_df], axis=1)
    final_exit_win_pc_df = pd.concat([final_exit_win_pc_df, exit_win_pc_df])

    core_win_pc_df = get_core_win_pc_df(core_walk_forward_dict, core_test_n_not_worked, n_total_cases)
    core_win_pc_df = pd.concat([temp_signal_df, core_win_pc_df], axis=1)
    final_core_win_pc_df = pd.concat([final_core_win_pc_df, core_win_pc_df])

    perf_df = get_perf_df(performance_walk_forward_dict, perf_n_not_worked, n_total_cases)
    perf_df = pd.concat([temp_signal_df, perf_df], axis=1)
    final_perf_df = pd.concat([final_perf_df, perf_df])

    mc_df = get_mc_df(mc_walk_forward_dict, mc_n_not_worked, n_total_cases)
    mc_df = pd.concat([temp_signal_df, mc_df], axis=1)
    final_mc_df = pd.concat([final_mc_df, mc_df])

    strategy_idx += 1

0it [00:00, ?it/s]

19532it [3:46:26,  1.44it/s]


In [26]:
final_entry_win_pc_df.reset_index(drop=True, inplace=True)
final_exit_win_pc_df.reset_index(drop=True, inplace=True)
final_core_win_pc_df.reset_index(drop=True, inplace=True) 
final_perf_df.reset_index(drop=True, inplace=True)
final_mc_df.reset_index(drop=True, inplace=True)

## Showing the entry testing results

In [27]:
final_entry_win_pc_df

,strategy,buy,sell,Fixed_StopLoss_TakeProfit_testing,Fixed_Bar_testing,Random_Exit_testing,Not_Working
0,strategy1,"(get_lag(price_data['xrp_volume'], lag=2)[MAX_...","((get_lag(price_data['ng_volume'], lag=5)[MAX_...",41.997174,65.487867,36.069703,0.0
1,strategy2,"(get_lag(price_data['xrp_volume'], lag=2)[MAX_...","((get_lag(price_data['ng_volume'], lag=5)[MAX_...",41.997174,65.487867,11.406867,0.0
2,strategy3,"(get_lag(price_data['xrp_volume'], lag=2)[MAX_...","((get_lag(price_data['ng_volume'], lag=5)[MAX_...",41.997174,65.487867,12.762342,0.0
3,strategy4,"(get_lag(price_data['xrp_volume'], lag=2)[MAX_...","((get_lag(price_data['ng_volume'], lag=5)[MAX_...",41.997174,65.487867,10.684084,0.0
4,strategy5,"(get_lag(price_data['xrp_volume'], lag=2)[MAX_...","((get_lag(price_data['ng_volume'], lag=5)[MAX_...",41.997174,65.487867,13.133216,0.0
...,...,...,...,...,...,...,...
19527,strategy19528,"(np.cos(get_lag(price_data['tia_open'], lag=2)...","(np.sqrt(get_lag(price_data['nq_volume'], lag=...",65.112732,64.202976,64.216666,0.0
19528,strategy19529,"get_lag(price_data['nvda_volume'], lag=2)[MAX_...","(np.sqrt(get_lag(price_data['amzn_high'], lag=...",41.959516,53.089755,53.353527,0.0
19529,strategy19530,"(get_lag(price_data['coin_volume'], lag=3)[MAX...","np.cos(get_lag(price_data['gc_high'], lag=4)[M...",30.551123,37.020746,34.877869,0.0
19530,strategy19531,np.cos(price_data['6e_low'][MAX_LAG:]) >= np.c...,(np.cos(price_data['tsla_open'][MAX_LAG:]) * n...,45.212853,52.879892,51.690544,0.0


### Finding strategies that have winning percentage equal to 60 or more for entry testing cases

In [28]:
entry_testing_strategies = final_entry_win_pc_df[
    (final_entry_win_pc_df['Fixed_StopLoss_TakeProfit_testing'] >= 60) & 
    (final_entry_win_pc_df['Fixed_Bar_testing'] >= 60) & 
    (final_entry_win_pc_df['Random_Exit_testing'] >= 60)
]['strategy'].tolist()
final_entry_win_pc_df[final_entry_win_pc_df['strategy'].isin(entry_testing_strategies)]

,strategy,buy,sell,Fixed_StopLoss_TakeProfit_testing,Fixed_Bar_testing,Random_Exit_testing,Not_Working
42,strategy43,"(get_lag(price_data['xrp_volume'], lag=2)[MAX_...","((get_lag(price_data['ng_volume'], lag=5)[MAX_...",74.716927,64.050374,76.153326,0.0
44,strategy45,"(get_lag(price_data['xrp_volume'], lag=2)[MAX_...","((get_lag(price_data['ng_volume'], lag=5)[MAX_...",74.716927,64.050374,73.752534,0.0
2218,strategy2219,"(get_lag(price_data['xrp_volume'], lag=2)[MAX_...","((get_lag(price_data['ng_volume'], lag=5)[MAX_...",74.641168,63.169858,64.083873,0.0
2219,strategy2220,"(get_lag(price_data['xrp_volume'], lag=2)[MAX_...","((get_lag(price_data['ng_volume'], lag=5)[MAX_...",74.641168,63.169858,75.339380,0.0
2223,strategy2224,"(get_lag(price_data['xrp_volume'], lag=2)[MAX_...","((get_lag(price_data['ng_volume'], lag=5)[MAX_...",74.641168,63.169858,76.248490,0.0
...,...,...,...,...,...,...,...
19455,strategy19456,numba_indicators.on_balance_volume(price=price...,"np.sin(get_lag(price_data['googl_open'], lag=1...",72.825842,72.939299,73.603241,0.0
19484,strategy19485,"np.sin(get_lag(price_data['aav_low'], lag=3)[M...","np.log(get_lag(price_data['sui_low'], lag=1)[M...",75.278715,93.780815,95.176138,0.0
19509,strategy19510,"(get_lag(price_data['inj_low'], lag=2)[MAX_LAG...","(np.sqrt(get_lag(price_data['dog_volume'], lag...",90.948144,97.350709,92.400532,0.0
19510,strategy19511,"(get_lag(price_data['inj_low'], lag=2)[MAX_LAG...","(np.sqrt(get_lag(price_data['dog_volume'], lag...",90.948144,97.350709,90.747295,0.0


## Showing exit testing results

In [29]:
final_exit_win_pc_df

,strategy,buy,sell,Trend_testing,Countertrend_testing,Random_Entry_testing,Not_Working
0,strategy1,"(get_lag(price_data['xrp_volume'], lag=2)[MAX_...","((get_lag(price_data['ng_volume'], lag=5)[MAX_...",60.354315,60.559432,48.275580,0.0
1,strategy2,"(get_lag(price_data['xrp_volume'], lag=2)[MAX_...","((get_lag(price_data['ng_volume'], lag=5)[MAX_...",60.354315,60.559432,49.323979,0.0
2,strategy3,"(get_lag(price_data['xrp_volume'], lag=2)[MAX_...","((get_lag(price_data['ng_volume'], lag=5)[MAX_...",60.354315,60.559432,49.267744,0.0
3,strategy4,"(get_lag(price_data['xrp_volume'], lag=2)[MAX_...","((get_lag(price_data['ng_volume'], lag=5)[MAX_...",60.354315,60.559432,49.441604,0.0
4,strategy5,"(get_lag(price_data['xrp_volume'], lag=2)[MAX_...","((get_lag(price_data['ng_volume'], lag=5)[MAX_...",60.354315,60.559432,49.398625,0.0
...,...,...,...,...,...,...,...
19527,strategy19528,"(np.cos(get_lag(price_data['tia_open'], lag=2)...","(np.sqrt(get_lag(price_data['nq_volume'], lag=...",60.356361,60.432567,48.397128,0.0
19528,strategy19529,"get_lag(price_data['nvda_volume'], lag=2)[MAX_...","(np.sqrt(get_lag(price_data['amzn_high'], lag=...",60.272487,60.350719,51.123113,0.0
19529,strategy19530,"(get_lag(price_data['coin_volume'], lag=3)[MAX...","np.cos(get_lag(price_data['gc_high'], lag=4)[M...",60.301127,60.809069,48.815612,0.0
19530,strategy19531,np.cos(price_data['6e_low'][MAX_LAG:]) >= np.c...,(np.cos(price_data['tsla_open'][MAX_LAG:]) * n...,60.033140,60.530785,49.938635,0.0


### Showing strategies that have passed entry testing

In [30]:
final_exit_win_pc_df[final_exit_win_pc_df['strategy'].isin(entry_testing_strategies)]

,strategy,buy,sell,Trend_testing,Countertrend_testing,Random_Entry_testing,Not_Working
42,strategy43,"(get_lag(price_data['xrp_volume'], lag=2)[MAX_...","((get_lag(price_data['ng_volume'], lag=5)[MAX_...",60.342041,60.475538,49.658389,0.0
44,strategy45,"(get_lag(price_data['xrp_volume'], lag=2)[MAX_...","((get_lag(price_data['ng_volume'], lag=5)[MAX_...",60.342041,60.475538,49.132712,0.0
2218,strategy2219,"(get_lag(price_data['xrp_volume'], lag=2)[MAX_...","((get_lag(price_data['ng_volume'], lag=5)[MAX_...",60.350224,60.559432,49.740212,0.0
2219,strategy2220,"(get_lag(price_data['xrp_volume'], lag=2)[MAX_...","((get_lag(price_data['ng_volume'], lag=5)[MAX_...",60.350224,60.559432,49.059035,0.0
2223,strategy2224,"(get_lag(price_data['xrp_volume'], lag=2)[MAX_...","((get_lag(price_data['ng_volume'], lag=5)[MAX_...",60.350224,60.559432,49.779051,0.0
...,...,...,...,...,...,...,...
19455,strategy19456,numba_indicators.on_balance_volume(price=price...,"np.sin(get_lag(price_data['googl_open'], lag=1...",60.337950,60.414152,49.483524,0.0
19484,strategy19485,"np.sin(get_lag(price_data['aav_low'], lag=3)[M...","np.log(get_lag(price_data['sui_low'], lag=1)[M...",60.305219,60.586033,49.523420,0.0
19509,strategy19510,"(get_lag(price_data['inj_low'], lag=2)[MAX_LAG...","(np.sqrt(get_lag(price_data['dog_volume'], lag...",60.325676,60.432567,48.547463,0.0
19510,strategy19511,"(get_lag(price_data['inj_low'], lag=2)[MAX_LAG...","(np.sqrt(get_lag(price_data['dog_volume'], lag...",60.325676,60.432567,49.578629,0.0


### Showing strategies that have winning percentage equal to 60 or more (except for Random_Entry_testing)

In [33]:
exit_testing_strategies = final_exit_win_pc_df[
    (final_exit_win_pc_df['Trend_testing'] >= 60) & 
    (final_exit_win_pc_df['Countertrend_testing'] >= 60) & 
    (final_exit_win_pc_df['Random_Entry_testing'] >= 50) &
    (final_exit_win_pc_df['strategy'].isin(entry_testing_strategies))
]['strategy'].tolist()

final_exit_win_pc_df[final_exit_win_pc_df['strategy'].isin(exit_testing_strategies)]

,strategy,buy,sell,Trend_testing,Countertrend_testing,Random_Entry_testing,Not_Working
2256,strategy2257,"(get_lag(price_data['xrp_volume'], lag=2)[MAX_...","((get_lag(price_data['ng_volume'], lag=5)[MAX_...",60.354315,60.559432,50.185113,0.0
2258,strategy2259,"(get_lag(price_data['xrp_volume'], lag=2)[MAX_...","((get_lag(price_data['ng_volume'], lag=5)[MAX_...",60.354315,60.559432,50.000000,0.0
2276,strategy2277,"(get_lag(price_data['xrp_volume'], lag=2)[MAX_...","((get_lag(price_data['ng_volume'], lag=5)[MAX_...",60.348178,60.559432,50.022499,0.0
2301,strategy2302,"(get_lag(price_data['xrp_volume'], lag=2)[MAX_...","((get_lag(price_data['ng_volume'], lag=5)[MAX_...",60.354315,60.559432,50.021480,0.0
2422,strategy2423,"(get_lag(price_data['xrp_volume'], lag=2)[MAX_...","((get_lag(price_data['ng_volume'], lag=5)[MAX_...",60.354315,60.559432,50.013298,0.0
...,...,...,...,...,...,...,...
19136,strategy19137,"(get_lag(price_data['ng_high'], lag=1)[MAX_LAG...","(get_lag(price_data['zn_volume'], lag=1)[MAX_L...",60.354315,60.426429,50.818197,0.0
19176,strategy19177,price_data['es_low'][MAX_LAG:] > get_lag(price...,numba_indicators.on_balance_volume(price=price...,60.315447,60.428475,50.093076,0.0
19329,strategy19330,"(get_lag(price_data['pltr_volume'], lag=4)[MAX...",price_data['nq_volume'][MAX_LAG:] >= price_dat...,60.368635,60.469399,50.120679,0.0
19348,strategy19349,price_data['ng_open'][MAX_LAG:] >= price_data[...,"get_lag(price_data['nq_high'], lag=3)[MAX_LAG:...",60.368635,60.432567,50.458265,0.0


## Showing core testing results

In [34]:
final_core_win_pc_df

,strategy,buy,sell,Core_Testing,Not_Working
0,strategy1,"(get_lag(price_data['xrp_volume'], lag=2)[MAX_...","((get_lag(price_data['ng_volume'], lag=5)[MAX_...",94.736842,0.0
1,strategy2,"(get_lag(price_data['xrp_volume'], lag=2)[MAX_...","((get_lag(price_data['ng_volume'], lag=5)[MAX_...",94.736842,0.0
2,strategy3,"(get_lag(price_data['xrp_volume'], lag=2)[MAX_...","((get_lag(price_data['ng_volume'], lag=5)[MAX_...",94.736842,0.0
3,strategy4,"(get_lag(price_data['xrp_volume'], lag=2)[MAX_...","((get_lag(price_data['ng_volume'], lag=5)[MAX_...",94.736842,0.0
4,strategy5,"(get_lag(price_data['xrp_volume'], lag=2)[MAX_...","((get_lag(price_data['ng_volume'], lag=5)[MAX_...",94.736842,0.0
...,...,...,...,...,...
19527,strategy19528,"(np.cos(get_lag(price_data['tia_open'], lag=2)...","(np.sqrt(get_lag(price_data['nq_volume'], lag=...",30.769231,0.0
19528,strategy19529,"get_lag(price_data['nvda_volume'], lag=2)[MAX_...","(np.sqrt(get_lag(price_data['amzn_high'], lag=...",24.827586,0.0
19529,strategy19530,"(get_lag(price_data['coin_volume'], lag=3)[MAX...","np.cos(get_lag(price_data['gc_high'], lag=4)[M...",28.415301,0.0
19530,strategy19531,np.cos(price_data['6e_low'][MAX_LAG:]) >= np.c...,(np.cos(price_data['tsla_open'][MAX_LAG:]) * n...,23.604466,0.0


### Showing strategies that has passed entry testing and have winning percentage equal to 60 or more

In [35]:
core_testing_strategies = final_core_win_pc_df[
    (final_core_win_pc_df['Core_Testing'] >= 60) &
    (final_core_win_pc_df['strategy'].isin(entry_testing_strategies))
]['strategy'].tolist()

final_core_win_pc_df[final_core_win_pc_df['strategy'].isin(core_testing_strategies)]

,strategy,buy,sell,Core_Testing,Not_Working
42,strategy43,"(get_lag(price_data['xrp_volume'], lag=2)[MAX_...","((get_lag(price_data['ng_volume'], lag=5)[MAX_...",94.117647,0.0
44,strategy45,"(get_lag(price_data['xrp_volume'], lag=2)[MAX_...","((get_lag(price_data['ng_volume'], lag=5)[MAX_...",94.117647,0.0
2218,strategy2219,"(get_lag(price_data['xrp_volume'], lag=2)[MAX_...","((get_lag(price_data['ng_volume'], lag=5)[MAX_...",93.333333,0.0
2219,strategy2220,"(get_lag(price_data['xrp_volume'], lag=2)[MAX_...","((get_lag(price_data['ng_volume'], lag=5)[MAX_...",93.333333,0.0
2223,strategy2224,"(get_lag(price_data['xrp_volume'], lag=2)[MAX_...","((get_lag(price_data['ng_volume'], lag=5)[MAX_...",93.333333,0.0
...,...,...,...,...,...
19348,strategy19349,price_data['ng_open'][MAX_LAG:] >= price_data[...,"get_lag(price_data['nq_high'], lag=3)[MAX_LAG:...",66.666667,0.0
19407,strategy19408,(numba_indicators.on_balance_volume(price=pric...,np.log(price_data['sui_volume'][MAX_LAG:]) < 5,63.157895,0.0
19439,strategy19440,"(get_lag(price_data['pltr_low'], lag=4)[MAX_LA...","get_lag(price_data['cl_volume'], lag=1)[MAX_LA...",66.666667,0.0
19441,strategy19442,"get_lag(price_data['gc_volume'], lag=3)[MAX_LA...","get_lag(price_data['zf_low'], lag=2)[MAX_LAG:]...",78.947368,0.0


### Showing strategies that has passed both entry and exit testing and have winning percentage equal to 60 or more

In [36]:
core_testing_strategies2 = final_core_win_pc_df[
    (final_core_win_pc_df['Core_Testing'] >= 60) &
    (final_core_win_pc_df['strategy'].isin(exit_testing_strategies))
]['strategy'].tolist()

final_core_win_pc_df[final_core_win_pc_df['strategy'].isin(core_testing_strategies2)]

,strategy,buy,sell,Core_Testing,Not_Working
2256,strategy2257,"(get_lag(price_data['xrp_volume'], lag=2)[MAX_...","((get_lag(price_data['ng_volume'], lag=5)[MAX_...",94.117647,0.0
2258,strategy2259,"(get_lag(price_data['xrp_volume'], lag=2)[MAX_...","((get_lag(price_data['ng_volume'], lag=5)[MAX_...",94.117647,0.0
2276,strategy2277,"(get_lag(price_data['xrp_volume'], lag=2)[MAX_...","((get_lag(price_data['ng_volume'], lag=5)[MAX_...",93.333333,0.0
2301,strategy2302,"(get_lag(price_data['xrp_volume'], lag=2)[MAX_...","((get_lag(price_data['ng_volume'], lag=5)[MAX_...",94.736842,0.0
2422,strategy2423,"(get_lag(price_data['xrp_volume'], lag=2)[MAX_...","((get_lag(price_data['ng_volume'], lag=5)[MAX_...",94.736842,0.0
...,...,...,...,...,...
18735,strategy18736,"np.sin(get_lag(price_data['btc_open'], lag=4)[...","(get_lag(price_data['tia_high'], lag=2)[MAX_LA...",60.000000,0.0
18982,strategy18983,"get_lag(price_data['amzn_low'], lag=5)[MAX_LAG...","(get_lag(price_data['zf_volume'], lag=4)[MAX_L...",72.727273,0.0
19136,strategy19137,"(get_lag(price_data['ng_high'], lag=1)[MAX_LAG...","(get_lag(price_data['zn_volume'], lag=1)[MAX_L...",71.428571,0.0
19176,strategy19177,price_data['es_low'][MAX_LAG:] > get_lag(price...,numba_indicators.on_balance_volume(price=price...,65.517241,0.0


## Showing performance

In [37]:
final_perf_df

,strategy,buy,sell,N_Trades,PNL,ROI (%),AVG_Drawdown,Drawdown (%),Drawdown_Duration,PNL_AVGD_Ratio,Sharpe_Ratio,Sortino_Ratio,Not_Working,MT_PNL_D,MT_MDD_D
0,strategy1,"(get_lag(price_data['xrp_volume'], lag=2)[MAX_...","((get_lag(price_data['ng_volume'], lag=5)[MAX_...",19.0,2.740325e+05,78.295001,7.435635,100.015647,13.0,36068.239247,372.058434,1.406236e+06,0.0,68.5750,37.0000
1,strategy2,"(get_lag(price_data['xrp_volume'], lag=2)[MAX_...","((get_lag(price_data['ng_volume'], lag=5)[MAX_...",19.0,2.740325e+05,78.295001,7.435635,100.015647,13.0,36068.239247,372.058434,1.406236e+06,0.0,68.2375,36.2250
2,strategy3,"(get_lag(price_data['xrp_volume'], lag=2)[MAX_...","((get_lag(price_data['ng_volume'], lag=5)[MAX_...",19.0,2.740325e+05,78.295001,7.435635,100.015647,13.0,36068.239247,372.058434,1.406236e+06,0.0,68.9000,36.6000
3,strategy4,"(get_lag(price_data['xrp_volume'], lag=2)[MAX_...","((get_lag(price_data['ng_volume'], lag=5)[MAX_...",19.0,2.740325e+05,78.295001,7.435635,100.015647,13.0,36068.239247,372.058434,1.406236e+06,0.0,68.4875,36.1500
4,strategy5,"(get_lag(price_data['xrp_volume'], lag=2)[MAX_...","((get_lag(price_data['ng_volume'], lag=5)[MAX_...",19.0,2.740325e+05,78.295001,7.435635,100.015647,13.0,36068.239247,372.058434,1.406236e+06,0.0,68.8625,36.1375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19527,strategy19528,"(np.cos(get_lag(price_data['tia_open'], lag=2)...","(np.sqrt(get_lag(price_data['nq_volume'], lag=...",13.0,7.354186e+04,21.011960,7389.820944,126.088783,8.0,1.516622,130.968815,4.151698e+02,0.0,58.2625,29.1625
19528,strategy19529,"get_lag(price_data['nvda_volume'], lag=2)[MAX_...","(np.sqrt(get_lag(price_data['amzn_high'], lag=...",145.0,-3.203073e+06,-915.163820,95404.242593,555.881390,91.0,1.514053,-698.731748,-4.408728e+02,0.0,0.0000,27.0875
19529,strategy19530,"(get_lag(price_data['coin_volume'], lag=3)[MAX...","np.cos(get_lag(price_data['gc_high'], lag=4)[M...",183.0,-8.472979e+03,-2.420851,13020.616864,182.583533,112.0,1.509539,-2.227615,-2.742130e+00,0.0,51.0500,40.3125
19530,strategy19531,np.cos(price_data['6e_low'][MAX_LAG:]) >= np.c...,(np.cos(price_data['tsla_open'][MAX_LAG:]) * n...,627.0,-1.245647e+06,-355.899223,10757.786738,367.547916,405.0,1.507837,-428.774061,-3.567462e+02,0.0,0.0000,40.1000


### Showing performance for strategies that has passed entry and core testing and have positive ROI

In [ ]:
# final_perf_df[
#     (final_perf_df['strategy'].isin(core_testing_strategies)) & 
#     (final_perf_df['ROI (%)'] > 0) &
#     (final_perf_df['MT_PNL_D'] > 90) &
#     (final_perf_df['MT_MDD_D'] > 90)
# ]

In [39]:
final_perf_df[
    (final_perf_df['strategy'].isin(core_testing_strategies)) & 
    (final_perf_df['ROI (%)'] > 0)
]

,strategy,buy,sell,N_Trades,PNL,ROI (%),AVG_Drawdown,Drawdown (%),Drawdown_Duration,PNL_AVGD_Ratio,Sharpe_Ratio,Sortino_Ratio,Not_Working,MT_PNL_D,MT_MDD_D
42,strategy43,"(get_lag(price_data['xrp_volume'], lag=2)[MAX_...","((get_lag(price_data['ng_volume'], lag=5)[MAX_...",17.0,274261.387612,78.360396,7.435635,100.015647,11.0,35426.664631,399.724975,1.572988e+06,0.0,69.4125,35.1750
44,strategy45,"(get_lag(price_data['xrp_volume'], lag=2)[MAX_...","((get_lag(price_data['ng_volume'], lag=5)[MAX_...",17.0,274261.387612,78.360396,7.435635,100.015647,11.0,35426.664631,399.724975,1.572988e+06,0.0,69.1625,35.4000
2218,strategy2219,"(get_lag(price_data['xrp_volume'], lag=2)[MAX_...","((get_lag(price_data['ng_volume'], lag=5)[MAX_...",15.0,261192.394880,74.626399,7.435635,100.015647,11.0,33605.928205,408.454656,1.697770e+06,0.0,70.2500,34.6000
2219,strategy2220,"(get_lag(price_data['xrp_volume'], lag=2)[MAX_...","((get_lag(price_data['ng_volume'], lag=5)[MAX_...",15.0,261192.394880,74.626399,7.435635,100.015647,11.0,33605.928205,408.454656,1.697770e+06,0.0,69.6500,33.6625
2223,strategy2224,"(get_lag(price_data['xrp_volume'], lag=2)[MAX_...","((get_lag(price_data['ng_volume'], lag=5)[MAX_...",15.0,261192.394880,74.626399,7.435635,100.015647,11.0,33605.928205,408.454656,1.697770e+06,0.0,69.4875,34.2125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19186,strategy19187,"(get_lag(price_data['zn_low'], lag=2)[MAX_LAG:...","np.sqrt(get_lag(price_data['fet_low'], lag=4)[...",45.0,115643.481892,33.040995,7770.993555,339.307853,11.0,2.981802,166.311053,1.414106e+02,0.0,56.6500,25.9250
19190,strategy19191,"(np.cos(get_lag(price_data['tia_low'], lag=1)[...","get_lag(price_data['dog_volume'], lag=1)[MAX_L...",5.0,86482.974537,24.709421,29825.908069,137.187969,4.0,2.956054,297.893587,4.204309e+02,0.0,63.3625,13.5125
19348,strategy19349,price_data['ng_open'][MAX_LAG:] >= price_data[...,"get_lag(price_data['nq_high'], lag=3)[MAX_LAG:...",3.0,111814.152526,31.946901,49520.052119,148.725886,2.0,2.257957,346.282165,5.456600e+02,0.0,72.0500,11.6500
19439,strategy19440,"(get_lag(price_data['pltr_low'], lag=4)[MAX_LA...","get_lag(price_data['cl_volume'], lag=1)[MAX_LA...",3.0,89761.922714,25.646264,47257.662338,362.806233,1.0,1.899415,258.890827,4.590144e+02,0.0,68.1875,6.3875


### Showing performance for strategies that has passed all tests and have positive ROI

In [40]:
final_perf_df[
    (final_perf_df['strategy'].isin(core_testing_strategies2)) & 
    (final_perf_df['ROI (%)'] > 0)
]

,strategy,buy,sell,N_Trades,PNL,ROI (%),AVG_Drawdown,Drawdown (%),Drawdown_Duration,PNL_AVGD_Ratio,Sharpe_Ratio,Sortino_Ratio,Not_Working,MT_PNL_D,MT_MDD_D
2256,strategy2257,"(get_lag(price_data['xrp_volume'], lag=2)[MAX_...","((get_lag(price_data['ng_volume'], lag=5)[MAX_...",17.0,262100.483855,74.885853,7.435635,100.015647,13.0,33582.190123,379.575972,1.503241e+06,0.0,69.3750,35.5875
2258,strategy2259,"(get_lag(price_data['xrp_volume'], lag=2)[MAX_...","((get_lag(price_data['ng_volume'], lag=5)[MAX_...",17.0,262100.483855,74.885853,7.435635,100.015647,13.0,33582.190123,379.575972,1.503241e+06,0.0,68.8375,34.4125
2276,strategy2277,"(get_lag(price_data['xrp_volume'], lag=2)[MAX_...","((get_lag(price_data['ng_volume'], lag=5)[MAX_...",15.0,256156.310889,73.187517,7.435635,100.015559,11.0,32370.747563,413.215618,1.665035e+06,0.0,69.2625,33.3000
2301,strategy2302,"(get_lag(price_data['xrp_volume'], lag=2)[MAX_...","((get_lag(price_data['ng_volume'], lag=5)[MAX_...",19.0,273351.821106,78.100520,8.365089,100.016618,13.0,31680.897670,368.812854,1.246882e+06,0.0,68.9625,36.6375
2422,strategy2423,"(get_lag(price_data['xrp_volume'], lag=2)[MAX_...","((get_lag(price_data['ng_volume'], lag=5)[MAX_...",19.0,270805.961319,77.373132,8.365089,100.017504,13.0,31376.554263,367.643752,1.235269e+06,0.0,67.4875,35.3000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18435,strategy18436,"get_lag(price_data['nq_volume'], lag=2)[MAX_LA...","get_lag(price_data['aav_volume'], lag=1)[MAX_L...",57.0,134394.871838,38.398535,3999.949186,144.626224,30.0,6.112011,138.753068,1.721399e+02,0.0,55.6375,39.3625
18735,strategy18736,"np.sin(get_lag(price_data['btc_open'], lag=4)[...","(get_lag(price_data['tia_high'], lag=2)[MAX_LA...",5.0,270967.145222,77.419184,54537.263123,116.739665,4.0,4.942161,255.702786,1.018440e+03,0.0,83.9125,14.7625
19136,strategy19137,"(get_lag(price_data['ng_high'], lag=1)[MAX_LAG...","(get_lag(price_data['zn_volume'], lag=1)[MAX_L...",7.0,181581.692262,51.880484,57412.730635,126.702937,6.0,3.189342,214.980281,4.559458e+02,0.0,71.9000,18.3500
19176,strategy19177,price_data['es_low'][MAX_LAG:] > get_lag(price...,numba_indicators.on_balance_volume(price=price...,29.0,6581.723946,1.880493,15234.350350,440.508440,18.0,3.002496,10.936588,7.275978e+00,0.0,50.6625,21.4250


In [41]:
# final_perf_df[
#     (final_perf_df['strategy'].isin(core_testing_strategies2)) & 
#     (final_perf_df['ROI (%)'] > 0)
# ][[col for col in final_perf_df.columns if col not in ['buy', 'sell']]]

In [42]:
# buy_str_list = final_perf_df[
#     (final_perf_df['strategy'].isin(core_testing_strategies2)) & 
#     (final_perf_df['ROI (%)'] > 0)
# ]['buy'].tolist()

# print(buy_str_list[3])

In [43]:
# sell_str_list = final_perf_df[
#     (final_perf_df['strategy'].isin(core_testing_strategies2)) & 
#     (final_perf_df['ROI (%)'] > 0)
# ]['sell'].tolist()

# print(sell_str_list[3])

### Showing Monte Carlo simulation

In [44]:
final_mc_df

,strategy,buy,sell,median_drawdown (%),median_drawdown_duration,median_profit,median_ROI (%),ratio,prob,Not_Working
0,strategy1,"(get_lag(price_data['xrp_volume'], lag=2)[MAX_...","((get_lag(price_data['ng_volume'], lag=5)[MAX_...",0.003725,1.0,2.613605e+05,74.674442,2.004747e+04,1.0000,0.0
1,strategy2,"(get_lag(price_data['xrp_volume'], lag=2)[MAX_...","((get_lag(price_data['ng_volume'], lag=5)[MAX_...",0.003674,1.0,2.629253e+05,75.121519,2.044518e+04,1.0000,0.0
2,strategy3,"(get_lag(price_data['xrp_volume'], lag=2)[MAX_...","((get_lag(price_data['ng_volume'], lag=5)[MAX_...",0.003556,1.0,2.598608e+05,74.245946,2.087803e+04,1.0000,0.0
3,strategy4,"(get_lag(price_data['xrp_volume'], lag=2)[MAX_...","((get_lag(price_data['ng_volume'], lag=5)[MAX_...",0.003555,1.0,2.592583e+05,74.073806,2.083524e+04,1.0000,0.0
4,strategy5,"(get_lag(price_data['xrp_volume'], lag=2)[MAX_...","((get_lag(price_data['ng_volume'], lag=5)[MAX_...",0.003605,1.0,2.617440e+05,74.783989,2.074382e+04,1.0000,0.0
...,...,...,...,...,...,...,...,...,...,...
19527,strategy19528,"(np.cos(get_lag(price_data['tia_open'], lag=2)...","(np.sqrt(get_lag(price_data['nq_volume'], lag=...",34.836945,7.0,6.114198e+04,17.469136,5.014543e-01,0.7018,0.0
19528,strategy19529,"get_lag(price_data['nvda_volume'], lag=2)[MAX_...","(np.sqrt(get_lag(price_data['amzn_high'], lag=...",0.000000,144.0,-3.210350e+06,-917.242795,-inf,0.0000,0.0
19529,strategy19530,"(get_lag(price_data['coin_volume'], lag=3)[MAX...","np.cos(get_lag(price_data['gc_high'], lag=4)[M...",335.896107,115.0,-9.414413e+03,-2.689832,-8.007929e-03,0.4833,0.0
19530,strategy19531,np.cos(price_data['6e_low'][MAX_LAG:]) >= np.c...,(np.cos(price_data['tsla_open'][MAX_LAG:]) * n...,0.000000,626.0,-1.244337e+06,-355.524779,-inf,0.0000,0.0


### Showing simulation results for strategies that has passed entry and core testing and have positive ROI

In [45]:
mc_strategies_list = final_perf_df[
    (final_perf_df['strategy'].isin(core_testing_strategies)) & 
    (final_perf_df['ROI (%)'] > 0)
]['strategy']

final_mc_df[final_mc_df['strategy'].isin(mc_strategies_list)]

,strategy,buy,sell,median_drawdown (%),median_drawdown_duration,median_profit,median_ROI (%),ratio,prob,Not_Working
42,strategy43,"(get_lag(price_data['xrp_volume'], lag=2)[MAX_...","((get_lag(price_data['ng_volume'], lag=5)[MAX_...",0.003583,1.0,261860.476460,74.817279,2.088307e+04,1.0000,0.0
44,strategy45,"(get_lag(price_data['xrp_volume'], lag=2)[MAX_...","((get_lag(price_data['ng_volume'], lag=5)[MAX_...",0.003601,1.0,262225.744445,74.921641,2.080677e+04,1.0000,0.0
2218,strategy2219,"(get_lag(price_data['xrp_volume'], lag=2)[MAX_...","((get_lag(price_data['ng_volume'], lag=5)[MAX_...",0.003641,1.0,249486.780014,71.281937,1.957836e+04,1.0000,0.0
2219,strategy2220,"(get_lag(price_data['xrp_volume'], lag=2)[MAX_...","((get_lag(price_data['ng_volume'], lag=5)[MAX_...",0.003772,1.0,248347.336915,70.956382,1.881373e+04,1.0000,0.0
2223,strategy2224,"(get_lag(price_data['xrp_volume'], lag=2)[MAX_...","((get_lag(price_data['ng_volume'], lag=5)[MAX_...",0.003753,1.0,249865.409249,71.390117,1.902124e+04,1.0000,0.0
...,...,...,...,...,...,...,...,...,...,...
19186,strategy19187,"(get_lag(price_data['zn_low'], lag=2)[MAX_LAG:...","np.sqrt(get_lag(price_data['fet_low'], lag=4)[...",75.315654,11.0,114246.885838,32.641967,4.334022e-01,0.9488,0.0
19190,strategy19191,"(np.cos(get_lag(price_data['tia_low'], lag=1)[...","get_lag(price_data['dog_volume'], lag=1)[MAX_L...",15.223316,1.0,86482.974537,24.709421,1.623130e+00,0.8620,0.0
19348,strategy19349,price_data['ng_open'][MAX_LAG:] >= price_data[...,"get_lag(price_data['nq_high'], lag=3)[MAX_LAG:...",0.000000,1.0,111814.152526,31.946901,inf,0.8521,0.0
19439,strategy19440,"(get_lag(price_data['pltr_low'], lag=4)[MAX_LA...","get_lag(price_data['cl_volume'], lag=1)[MAX_LA...",0.000000,1.0,89761.922714,25.646264,inf,0.7400,0.0


### Showing simulation results for strategies that has passed all tests and have positive ROI

In [46]:
mc_strategies_list2 = final_perf_df[
    (final_perf_df['strategy'].isin(core_testing_strategies2)) & 
    (final_perf_df['ROI (%)'] > 0)
]['strategy']

final_mc_df[final_mc_df['strategy'].isin(mc_strategies_list2)]

,strategy,buy,sell,median_drawdown (%),median_drawdown_duration,median_profit,median_ROI (%),ratio,prob,Not_Working
2256,strategy2257,"(get_lag(price_data['xrp_volume'], lag=2)[MAX_...","((get_lag(price_data['ng_volume'], lag=5)[MAX_...",0.003749,1.0,249279.948831,71.222843,1.899771e+04,1.0000,0.0
2258,strategy2259,"(get_lag(price_data['xrp_volume'], lag=2)[MAX_...","((get_lag(price_data['ng_volume'], lag=5)[MAX_...",0.003539,1.0,251109.492777,71.745569,2.027071e+04,1.0000,0.0
2276,strategy2277,"(get_lag(price_data['xrp_volume'], lag=2)[MAX_...","((get_lag(price_data['ng_volume'], lag=5)[MAX_...",0.003555,1.0,244545.555687,69.870159,1.965356e+04,1.0000,0.0
2301,strategy2302,"(get_lag(price_data['xrp_volume'], lag=2)[MAX_...","((get_lag(price_data['ng_volume'], lag=5)[MAX_...",0.004254,1.0,256695.510337,73.341574,1.723934e+04,1.0000,0.0
2422,strategy2423,"(get_lag(price_data['xrp_volume'], lag=2)[MAX_...","((get_lag(price_data['ng_volume'], lag=5)[MAX_...",0.004019,1.0,258307.539621,73.802154,1.836271e+04,1.0000,0.0
...,...,...,...,...,...,...,...,...,...,...
18435,strategy18436,"get_lag(price_data['nq_volume'], lag=2)[MAX_LA...","get_lag(price_data['aav_volume'], lag=1)[MAX_L...",68.120198,13.0,127947.243554,36.556355,5.366449e-01,0.9473,0.0
18735,strategy18736,"np.sin(get_lag(price_data['btc_open'], lag=4)[...","(get_lag(price_data['tia_high'], lag=2)[MAX_LA...",8.354563,2.0,270967.145222,77.419184,9.266694e+00,0.7101,0.0
19136,strategy19137,"(get_lag(price_data['ng_high'], lag=1)[MAX_LAG...","(get_lag(price_data['zn_volume'], lag=1)[MAX_L...",26.702937,2.0,170340.008152,48.668574,1.822593e+00,0.7514,0.0
19176,strategy19177,price_data['es_low'][MAX_LAG:] > get_lag(price...,numba_indicators.on_balance_volume(price=price...,184.551762,14.0,13640.966709,3.897419,2.111830e-02,0.5678,0.0


In [42]:
final_entry_win_pc_df.to_csv('testing_results/entry_testing_last.csv', index=False)
final_exit_win_pc_df.to_csv('testing_results/exit_testing_last.csv', index=False)
final_core_win_pc_df.to_csv('testing_results/core_testing_last.csv', index=False)
final_perf_df.to_csv('testing_results/perf_last.csv', index=False)
final_mc_df.to_csv('testing_results/mc_last.csv', index=False)

# Showing equity curves

In [56]:
# plt.plot(equity_curve_dict[19532][0]);